In [1]:
import pandas as pd
import pyodbc

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

In [2]:
connection = pyodbc.connect(
  r'Driver={ODBC Driver 17 for SQL Server};'
  r'Server=localhost;'
  r'Database=AdventureWorks2022;'
  r'Trusted_Connection=yes;'
)

In [3]:
#class for reading table
class db_reader:
    def read_table(self, table_name, connection, col=None):
        if col:
            query = f"SELECT {', '.join(col)} FROM {table_name}" #', '.join(col) converts the list of columns into a comma-separated string. For example, if col = ['name', 'salary'], ', '.join(col) produces 'name, salary'.
        else:
            query = f"SELECT * FROM {table_name}"
        return pd.read_sql_query(query, connection)

In [4]:
#class for merging
class merge:
  def merge_tbl(self, t1, t2, join, col = None, col1 = None, col2 = None):
    if col1 and col2:
      return pd.merge(t1, t2, how = join, left_on = col1, right_on = col2)
    elif col:
      return pd.merge(t1, t2, how = join, on = col)

In [5]:
# 1 Query to find the average sales by year: Write a SQL query to find the average sales for each year from the Sales SalesOrderHeader table

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [6]:
soh.groupby(pd.to_datetime(soh['OrderDate']).dt.year)['TotalDue'].mean().rename('avg').reset_index()

,OrderDate,avg
0,2011,8808.773818
1,2012,9623.422813
2,2013,3452.678604
3,2014,1906.257828


In [7]:
# 2 Query to find the total sales by year and month: Write a SQL query to find the total sales for each year and month from the Sales SalesOrderHeader table

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()


,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [8]:
soh.groupby([pd.to_datetime(soh['OrderDate']).dt.year.rename('Year'), pd.to_datetime(soh['OrderDate']).dt.month.rename('Month')])['TotalDue'].sum().rename('sum').reset_index().head()

,Year,Month,sum
0,2011,5,5.670209e+05
1,2011,6,5.070965e+05
2,2011,7,2.292183e+06
3,2011,8,2.800576e+06
4,2011,9,5.547916e+05


In [9]:
# 3 Query to find the products that have never been sold: Write a SQL query to find the products that have never been sold from the Sales SalesOrderDetail and Production Product tables

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [10]:
sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [11]:
p[~p['ProductID'].isin(sod['ProductID'])].head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [12]:
# 4 Query to find the customers who have not placed any orders in the last year: Write a SQL query to find the customers who have not placed any orders in the last year from the Sales SalesOrderHeader table

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [13]:
last_year = max(soh['OrderDate']) - pd.DateOffset(years=1)

In [14]:
soh[pd.to_datetime(soh['OrderDate']) > last_year].head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
8262,51922,8,2013-07-01,2013-07-13,2013-07-08,5,True,SO51922,None,10-4030-024611,24611,NaN,9,20799,20799,1,18270.0,416142Vi94824,8752.0,1218.98,97.5184,30.4745,1346.9729,None,8F4CD653-F676-415C-A4DC-7D1E07BF10E4,2013-07-08
8263,51923,8,2013-07-01,2013-07-13,2013-07-08,5,True,SO51923,None,10-4030-011420,11420,NaN,7,25115,25115,1,12329.0,716484Vi63799,NaN,2376.96,190.1568,59.4240,2626.5408,None,8D3A27F8-5363-41BE-93D1-BF92D6001E66,2013-07-08
8264,51924,8,2013-07-01,2013-07-13,2013-07-08,5,True,SO51924,None,10-4030-028607,28607,NaN,1,15116,15116,1,10455.0,1016733Vi53931,NaN,7.95,0.6360,0.1988,8.7848,None,E9F9D42E-4384-4375-8614-270298A617AB,2013-07-08
8265,51925,8,2013-07-01,2013-07-13,2013-07-08,5,True,SO51925,None,10-4030-011091,11091,NaN,6,13550,13550,1,11334.0,916743Vi58449,8754.0,13.98,1.1184,0.3495,15.4479,None,74043B7D-345B-454D-ABC9-D16FC9F5F79A,2013-07-08
8266,51926,8,2013-07-01,2013-07-13,2013-07-08,5,True,SO51926,None,10-4030-014557,14557,NaN,9,23619,23619,1,16302.0,517101Vi84279,8752.0,63.97,5.1176,1.5993,70.6869,None,DF34335B-6E5D-4883-87AF-1A442B555377,2013-07-08


In [15]:
# 5 Query to find the most popular product category: Write a SQL query to find the product category that has the highest total sales from the Sales SalesOrderDetail, Production Product, and Production ProductSubcategory tables

sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [16]:
p = db_reader().read_table('Production.Product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [17]:
sal = merge().merge_tbl(sod[['ProductID', 'LineTotal']], p[['ProductID', 'ProductSubcategoryID']], 'inner', 'ProductID')
sal.head()

,ProductID,LineTotal,ProductSubcategoryID
0,762,419.4589,2.0
1,758,874.7940,2.0
2,745,809.7600,12.0
3,743,714.7043,12.0
4,747,1429.4086,12.0


In [18]:
psc = db_reader().read_table('Production.Productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [19]:
cat = merge().merge_tbl(sal, psc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner', 'ProductSubcategoryID')
cat.head()

,ProductID,LineTotal,ProductSubcategoryID,ProductCategoryID
0,762,419.4589,2.0,1
1,758,874.7940,2.0,1
2,745,809.7600,12.0,2
3,743,714.7043,12.0,2
4,747,1429.4086,12.0,2


In [20]:
cat.groupby('ProductCategoryID')['LineTotal'].sum().reset_index().sort_values('LineTotal', ascending=False)

,ProductCategoryID,LineTotal
0,1,9.463085e+07
1,2,1.180259e+07
2,3,2.120383e+06
3,4,1.271992e+06


In [21]:
# 6 Query to find the salesperson with the highest total sales: Write a SQL query to find the salesperson who has the highest total sales from the Sales SalesOrderHeader and Sales SalesPerson tables

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [22]:
soh.groupby('SalesPersonID')['TotalDue'].sum().rename('Total').reset_index().sort_values('Total', ascending=False).head()

,SalesPersonID,Total
2,276.0,1.169502e+07
3,277.0,1.134239e+07
1,275.0,1.047537e+07
15,289.0,9.585125e+06
5,279.0,8.062920e+06


In [23]:
# 7 Query to find the total quantity of each product sold: Write a SQL query to find the total quantity of each product sold from the Sales SalesOrderDetail table

sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [24]:
sod.groupby('ProductID')['OrderQty'].sum().rename('count_of_products').reset_index().head()

,ProductID,count_of_products
0,707,6266
1,708,6532
2,709,1101
3,710,90
4,711,6739


In [25]:
# 8 Query to find the average sales amount for each salesperson: Write a SQL query to find the average sales amount for each salesperson from the Sales SalesOrderHeader and Sales SalesPerson tables

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [26]:
sp = db_reader().read_table('sales.customer', connection)
sp.head()

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,2,NaN,1028.0,1,AW00000002,E552F657-A9AF-4A7D-A645-C429D6E02491,2014-09-12 11:15:07.263
1,3,NaN,642.0,4,AW00000003,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2014-09-12 11:15:07.263
2,4,NaN,932.0,4,AW00000004,FF862851-1DAA-4044-BE7C-3E85583C054D,2014-09-12 11:15:07.263
3,5,NaN,1026.0,4,AW00000005,83905BDC-6F5E-4F71-B162-C98DA069F38A,2014-09-12 11:15:07.263
4,6,NaN,644.0,4,AW00000006,1A92DF88-BFA2-467D-BD54-FCB9E647FDD7,2014-09-12 11:15:07.263


In [27]:
per = merge().merge_tbl(soh[['CustomerID', 'TotalDue']], sp['PersonID'], 'inner', col1 = 'CustomerID',col2 = 'PersonID')
per.head()

,CustomerID,TotalDue,PersonID
0,14501,772.5036,14501.0
1,11003,3756.9890,11003.0
2,16624,3953.9884,16624.0
3,11005,3729.3640,11005.0
4,11011,3756.9890,11011.0


In [28]:
per.groupby('CustomerID')['TotalDue'].mean().reset_index().head()

,CustomerID,TotalDue
0,11000,3038.378033
1,11001,2351.395833
2,11002,2988.671433
3,11003,2997.971833
4,11004,3018.863700


In [29]:
# 9 Query to find the customers who have placed more than 10 orders: Write a SQL query to find the customers who have placed more than 10 orders from the Sales SalesOrderHeader table

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [30]:
soh.groupby('CustomerID')['SalesOrderID'].count().rename('count').reset_index().query('count > 10').head()

,CustomerID,count
19,11019,17
78,11078,17
91,11091,28
142,11142,17
176,11176,28


In [31]:
# 10 Query to find the most expensive product in each category: Write a SQL query to find the most expensive product in each category from the Production Product and Production ProductSubcategory tables

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [32]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [33]:
cat = merge().merge_tbl(p[['ProductSubcategoryID', 'ProductID', 'Name', 'ListPrice']], psc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner', 'ProductSubcategoryID')
cat.head()

,ProductSubcategoryID,ProductID,Name,ListPrice,ProductCategoryID
0,14.0,680,"HL Road Frame - Black, 58",1431.50,2
1,14.0,706,"HL Road Frame - Red, 58",1431.50,2
2,31.0,707,"Sport-100 Helmet, Red",34.99,4
3,31.0,708,"Sport-100 Helmet, Black",34.99,4
4,23.0,709,"Mountain Bike Socks, M",9.50,3


In [34]:
max = cat.groupby(['ProductCategoryID'])['ListPrice'].max().reset_index()
max

,ProductCategoryID,ListPrice
0,1,146.674
1,2,1431.500
2,3,89.990
3,4,159.000


In [35]:
cat[cat['ListPrice'].isin(max['ListPrice'])].head(10)

,ProductSubcategoryID,ProductID,Name,ListPrice,ProductCategoryID
0,14.0,680,"HL Road Frame - Black, 58",1431.500,2
1,14.0,706,"HL Road Frame - Red, 58",1431.500,2
12,14.0,717,"HL Road Frame - Red, 62",1431.500,2
13,14.0,718,"HL Road Frame - Red, 44",1431.500,2
14,14.0,719,"HL Road Frame - Red, 48",1431.500,2
15,14.0,720,"HL Road Frame - Red, 52",1431.500,2
16,14.0,721,"HL Road Frame - Red, 56",1431.500,2
44,2.0,749,"Road-150 Red, 62",146.674,1
45,2.0,750,"Road-150 Red, 44",146.674,1
46,2.0,751,"Road-150 Red, 48",146.674,1


In [36]:
# 11 Query to find the total sales for each territory: Write a SQL query to find the total sales for each territory from the Sales SalesOrderHeader and Sales SalesTerritory tables

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [37]:
st = db_reader().read_table('sales.SalesTerritory', connection)
st.head()

,TerritoryID,Name,CountryRegionCode,Group,SalesYTD,SalesLastYear,CostYTD,CostLastYear,rowguid,ModifiedDate
0,1,Northwest,US,North America,7.887187e+06,3.298694e+06,0.0,0.0,43689A10-E30B-497F-B0DE-11DE20267FF7,2008-04-30
1,2,Northeast,US,North America,2.402177e+06,3.607149e+06,0.0,0.0,00FB7309-96CC-49E2-8363-0A1BA72486F2,2008-04-30
2,3,Central,US,North America,3.072175e+06,3.205014e+06,0.0,0.0,DF6E7FD8-1A8D-468C-B103-ED8ADDB452C1,2008-04-30
3,4,Southwest,US,North America,1.051085e+07,5.366576e+06,0.0,0.0,DC3E9EA0-7950-4431-9428-99DBCBC33865,2008-04-30
4,5,Southeast,US,North America,2.538667e+06,3.925071e+06,0.0,0.0,6DC4165A-5E4C-42D2-809D-4344E0AC75E7,2008-04-30


In [38]:
sal = merge().merge_tbl(soh[['TotalDue','TerritoryID']], st[['TerritoryID']], 'inner', 'TerritoryID')
sal.head()

,TotalDue,TerritoryID
0,1457.3288,5
1,36865.8012,6
2,32474.9324,6
3,472.3108,4
4,27510.4109,1


In [39]:
sal.groupby('TerritoryID')['TotalDue'].sum().reset_index()

,TerritoryID,TotalDue
0,1,1.806166e+07
1,2,7.820210e+06
2,3,8.913299e+06
3,4,2.715059e+07
4,5,8.884099e+06
5,6,1.839893e+07
6,7,8.119749e+06
7,8,5.479820e+06
8,9,1.181438e+07
9,10,8.574049e+06


In [40]:
# 12 Query to find the total number of employees in each department: Write a SQL query to find the total number of employees in each department from the HumanResources EmployeeDepartmentHistory table

edh = db_reader().read_table('HumanResources.EmployeeDepartmentHistory', connection)
edh.head()

,BusinessEntityID,DepartmentID,ShiftID,StartDate,EndDate,ModifiedDate
0,1,16,1,2009-01-14,None,2009-01-13
1,2,1,1,2008-01-31,None,2008-01-30
2,3,1,1,2007-11-11,None,2007-11-10
3,4,1,1,2007-12-05,2010-05-30,2010-05-28
4,4,2,1,2010-05-31,None,2010-05-30


In [41]:
edh[edh['EndDate'].isnull()].groupby('DepartmentID')['BusinessEntityID'].count().reset_index()

,DepartmentID,BusinessEntityID
0,1,6
1,2,4
2,3,18
3,4,9
4,5,12
5,6,4
6,7,179
7,8,6
8,9,6
9,10,10


In [42]:
# 13 Query to find the total sales by each customer: Write a SQL query to find the total sales by each customer from the Sales SalesOrderHeader table

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [43]:
soh.groupby('CustomerID')['TotalDue'].sum().reset_index().head()

,CustomerID,TotalDue
0,11000,9115.1341
1,11001,7054.1875
2,11002,8966.0143
3,11003,8993.9155
4,11004,9056.5911


In [44]:
# 14 Query to find the most popular product category: Write a SQL query to find the most popular product category based on the total quantity of products sold from the Sales SalesOrderDetail and Production Product tables

sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [45]:
p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [46]:
sal = merge().merge_tbl(sod[['ProductID', 'OrderQty']], p[['ProductID', 'ProductSubcategoryID']], 'inner', 'ProductID')
sal.head()

,ProductID,OrderQty,ProductSubcategoryID
0,762,1,2.0
1,758,1,2.0
2,745,1,12.0
3,743,1,12.0
4,747,2,12.0


In [47]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [48]:
cat = merge().merge_tbl(sal, psc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner', 'ProductSubcategoryID')
cat.head()

,ProductID,OrderQty,ProductSubcategoryID,ProductCategoryID
0,762,1,2.0,1
1,758,1,2.0,1
2,745,1,12.0,2
3,743,1,12.0,2
4,747,2,12.0,2


In [49]:
cat.groupby('ProductCategoryID')['OrderQty'].sum().reset_index()

,ProductCategoryID,OrderQty
0,1,90258
1,2,49044
2,3,73658
3,4,61928


In [50]:
# 15 Query to find the total purchase order amount for each vendor: Write a SQL query to find the total purchase order amount for each vendor from the Purchasing PurchaseOrderHeader table

ven = db_reader().read_table('Purchasing.PurchaseOrderHeader', connection)
ven.head()

,PurchaseOrderID,RevisionNumber,Status,EmployeeID,VendorID,ShipMethodID,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate
0,1,4,4,258,1580,3,2011-04-16,2011-04-25,201.0400,16.0832,5.0260,222.1492,2011-04-25
1,2,4,1,254,1496,5,2011-04-16,2011-04-25,272.1015,21.7681,6.8025,300.6721,2011-04-25
2,3,4,4,257,1494,2,2011-04-16,2011-04-25,8847.3000,707.7840,221.1825,9776.2665,2011-04-25
3,4,4,3,261,1650,5,2011-04-16,2011-04-25,171.0765,13.6861,4.2769,189.0395,2011-04-25
4,5,4,4,251,1654,4,2011-04-30,2011-05-09,20397.3000,1631.7840,509.9325,22539.0165,2011-05-09


In [51]:
ven.groupby('VendorID')['TotalDue'].sum().reset_index().head()

,VendorID,TotalDue
0,1492,2.506004e+04
1,1494,4.985896e+05
2,1496,2.850209e+04
3,1498,1.461654e+06
4,1500,2.314699e+04


In [52]:
# 16 Query to find the total number of work orders for each product: Write a SQL query to find the total number of work orders for each product from the Production WorkOrder table

wo = db_reader().read_table('Production.WorkOrder', connection)
wo.head()

,WorkOrderID,ProductID,OrderQty,StockedQty,ScrappedQty,StartDate,EndDate,DueDate,ScrapReasonID,ModifiedDate
0,1,722,8,8,0,2011-06-03,2011-06-13,2011-06-14,NaN,2011-06-13
1,2,725,15,15,0,2011-06-03,2011-06-13,2011-06-14,NaN,2011-06-13
2,3,726,9,9,0,2011-06-03,2011-06-13,2011-06-14,NaN,2011-06-13
3,4,729,16,16,0,2011-06-03,2011-06-13,2011-06-14,NaN,2011-06-13
4,5,730,14,14,0,2011-06-03,2011-06-13,2011-06-14,NaN,2011-06-13


In [53]:
wo.groupby('ProductID')['WorkOrderID'].count().reset_index().head()

,ProductID,WorkOrderID
0,3,1093
1,316,1093
2,324,1093
3,327,1093
4,328,926


In [54]:
#17 Query to find the top 5 products with the highest total sales: Write a SQL query to find the top 5 products with the highest total sales from the Sales SalesOrderDetail table 

sod = db_reader().read_table('Sales.SalesOrderDetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [55]:
sod.groupby('ProductID')['LineTotal'].sum().reset_index().sort_values('LineTotal', ascending=False).head()

,ProductID,LineTotal
65,782,4.400593e+06
66,783,4.009495e+06
62,779,3.693678e+06
63,780,3.438479e+06
64,781,3.434257e+06


In [56]:
# 18 Query to find the total sales by year and by product category: Write a SQL query to find the total sales by year and by product category from the Sales SalesOrderHeader, Sales SalesOrderDetail, and Production Product tables 

sod = db_reader().read_table('Sales.SalesOrderDetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [57]:
soh = db_reader().read_table('Sales.SalesOrderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [58]:
sale = merge().merge_tbl(sod[['SalesOrderID','ProductID']], soh[['SalesOrderID', 'TotalDue', 'OrderDate']], 'inner', 'SalesOrderID')
sale.head()

,SalesOrderID,ProductID,TotalDue,OrderDate
0,43660,762,1457.3288,2011-05-20
1,43660,758,1457.3288,2011-05-20
2,43661,745,36865.8012,2011-05-31
3,43661,743,36865.8012,2011-05-31
4,43661,747,36865.8012,2011-05-31


In [59]:
p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [60]:
sc = merge().merge_tbl(sale, p[['ProductID', 'ProductSubcategoryID']], 'inner','ProductID')
sc.head()

,SalesOrderID,ProductID,TotalDue,OrderDate,ProductSubcategoryID
0,43660,762,1457.3288,2011-05-20,2.0
1,43660,758,1457.3288,2011-05-20,2.0
2,43661,745,36865.8012,2011-05-31,12.0
3,43661,743,36865.8012,2011-05-31,12.0
4,43661,747,36865.8012,2011-05-31,12.0


In [61]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [62]:
cat = merge().merge_tbl(sc, psc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner', 'ProductSubcategoryID')
cat.head()

,SalesOrderID,ProductID,TotalDue,OrderDate,ProductSubcategoryID,ProductCategoryID
0,43660,762,1457.3288,2011-05-20,2.0,1
1,43660,758,1457.3288,2011-05-20,2.0,1
2,43661,745,36865.8012,2011-05-31,12.0,2
3,43661,743,36865.8012,2011-05-31,12.0,2
4,43661,747,36865.8012,2011-05-31,12.0,2


In [63]:
cat.groupby([pd.to_datetime(cat['OrderDate']).dt.year, 'ProductCategoryID'])['TotalDue'].sum().reset_index()

,OrderDate,ProductCategoryID,TotalDue
0,2011,1,9.417269e+07
1,2011,2,3.654793e+07
2,2011,3,2.497230e+07
3,2011,4,1.481298e+07
4,2012,1,4.106623e+08
5,2012,2,2.970518e+08
6,2012,3,1.958799e+08
7,2012,4,6.918332e+07
8,2013,1,5.068593e+08
9,2013,2,4.594074e+08


In [64]:
# 19 Query to find the total quantity of products sold by each salesperson: Write a SQL query to find the total quantity of products sold by each salesperson from the Sales SalesOrderHeader, Sales SalesOrderDetail, and Sales SalesPerson tables 

sod = db_reader().read_table('Sales.SalesOrderDetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [65]:
soh = db_reader().read_table('Sales.SalesOrderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [66]:
sale = merge().merge_tbl(sod[['SalesOrderID','ProductID','OrderQty']], soh[['SalesOrderID', 'SalesPersonID','TotalDue', 'OrderDate']], 'inner', 'SalesOrderID')
sale.head()

,SalesOrderID,ProductID,OrderQty,SalesPersonID,TotalDue,OrderDate
0,43660,762,1,279.0,1457.3288,2011-05-20
1,43660,758,1,279.0,1457.3288,2011-05-20
2,43661,745,1,282.0,36865.8012,2011-05-31
3,43661,743,1,282.0,36865.8012,2011-05-31
4,43661,747,2,282.0,36865.8012,2011-05-31


In [67]:
sp = db_reader().read_table('sales.salesperson', connection)
sp.head()

,BusinessEntityID,TerritoryID,SalesQuota,Bonus,CommissionPct,SalesYTD,SalesLastYear,rowguid,ModifiedDate
0,274,NaN,NaN,0.0,0.000,5.596976e+05,0.000000e+00,48754992-9EE0-4C0E-8C94-9451604E3E02,2010-12-28
1,275,2.0,300000.0,4100.0,0.012,3.763178e+06,1.750406e+06,1E0A7274-3064-4F58-88EE-4C6586C87169,2011-05-24
2,276,4.0,250000.0,2000.0,0.015,4.251369e+06,1.439156e+06,4DD9EEE4-8E81-4F8C-AF97-683394C1F7C0,2011-05-24
3,277,3.0,250000.0,2500.0,0.015,3.189418e+06,1.997186e+06,39012928-BFEC-4242-874D-423162C3F567,2011-05-24
4,278,6.0,250000.0,500.0,0.010,1.453719e+06,1.620277e+06,7A0AE1AB-B283-40F9-91D1-167ABF06D720,2011-05-24


In [68]:
per = merge().merge_tbl(sale, sp['BusinessEntityID'], 'inner', col1 = 'SalesPersonID',col2 = 'BusinessEntityID')
per.head()

,SalesOrderID,ProductID,OrderQty,SalesPersonID,TotalDue,OrderDate,BusinessEntityID
0,43660,762,1,279.0,1457.3288,2011-05-20,279
1,43660,758,1,279.0,1457.3288,2011-05-20,279
2,43661,745,1,282.0,36865.8012,2011-05-31,282
3,43661,743,1,282.0,36865.8012,2011-05-31,282
4,43661,747,2,282.0,36865.8012,2011-05-31,282


In [69]:
per.groupby('BusinessEntityID')['OrderQty'].sum().reset_index().head()

,BusinessEntityID,OrderQty
0,274,3095
1,275,23058
2,276,27229
3,277,27051
4,278,11544


In [70]:
# 20 Query to find the total purchase order amount for each product by year: Write a SQL query to find the total purchase order amount for each product by year from the Purchasing PurchaseOrderHeader, Purchasing PurchaseOrderDetail, and Production Product tables 

poh = db_reader().read_table('Purchasing.PurchaseOrderHeader', connection)
poh.head()

,PurchaseOrderID,RevisionNumber,Status,EmployeeID,VendorID,ShipMethodID,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate
0,1,4,4,258,1580,3,2011-04-16,2011-04-25,201.0400,16.0832,5.0260,222.1492,2011-04-25
1,2,4,1,254,1496,5,2011-04-16,2011-04-25,272.1015,21.7681,6.8025,300.6721,2011-04-25
2,3,4,4,257,1494,2,2011-04-16,2011-04-25,8847.3000,707.7840,221.1825,9776.2665,2011-04-25
3,4,4,3,261,1650,5,2011-04-16,2011-04-25,171.0765,13.6861,4.2769,189.0395,2011-04-25
4,5,4,4,251,1654,4,2011-04-30,2011-05-09,20397.3000,1631.7840,509.9325,22539.0165,2011-05-09


In [71]:
pod = db_reader().read_table('Purchasing.PurchaseOrderDetail', connection)
pod.head()

,PurchaseOrderID,PurchaseOrderDetailID,DueDate,OrderQty,ProductID,UnitPrice,LineTotal,ReceivedQty,RejectedQty,StockedQty,ModifiedDate
0,1,1,2011-04-30,4,1,50.2600,201.0400,3.0,0.0,3.0,2011-04-23
1,2,2,2011-04-30,3,359,45.1200,135.3600,3.0,0.0,3.0,2011-04-23
2,2,3,2011-04-30,3,360,45.5805,136.7415,3.0,0.0,3.0,2011-04-23
3,3,4,2011-04-30,550,530,16.0860,8847.3000,550.0,0.0,550.0,2011-04-23
4,4,5,2011-04-30,3,4,57.0255,171.0765,2.0,1.0,1.0,2011-04-23


In [72]:
pur = merge().merge_tbl(poh[['PurchaseOrderID', 'OrderDate', 'TotalDue']], pod[['PurchaseOrderID', 'ProductID']],'inner' ,'PurchaseOrderID')
pur.head()

,PurchaseOrderID,OrderDate,TotalDue,ProductID
0,1,2011-04-16,222.1492,1
1,2,2011-04-16,300.6721,359
2,2,2011-04-16,300.6721,360
3,3,2011-04-16,9776.2665,530
4,4,2011-04-16,189.0395,4


In [73]:
p = db_reader().read_table('Production.Product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [74]:
prod = merge().merge_tbl(pur, p[['ProductID', 'Name']], 'inner', 'ProductID')
prod.head()

,PurchaseOrderID,OrderDate,TotalDue,ProductID,Name
0,1,2011-04-16,222.1492,1,Adjustable Race
1,2,2011-04-16,300.6721,359,Thin-Jam Hex Nut 9
2,2,2011-04-16,300.6721,360,Thin-Jam Hex Nut 10
3,3,2011-04-16,9776.2665,530,Seat Post
4,4,2011-04-16,189.0395,4,Headset Ball Bearings


In [75]:
prod.groupby([pd.to_datetime(prod['OrderDate']).dt.year,'ProductID'])['TotalDue'].sum().reset_index().head()

,OrderDate,ProductID,TotalDue
0,2011,1,222.1492
1,2011,4,189.0395
2,2011,317,64847.5328
3,2011,318,64847.5328
4,2011,319,64847.5328


In [76]:
# 21 Query to find the total number of work orders for each product by year: Write a SQL query to find the total number of work orders for each product by year from the Production WorkOrder table 

wo = db_reader().read_table('Production.WorkOrder', connection)
wo.head()

,WorkOrderID,ProductID,OrderQty,StockedQty,ScrappedQty,StartDate,EndDate,DueDate,ScrapReasonID,ModifiedDate
0,1,722,8,8,0,2011-06-03,2011-06-13,2011-06-14,NaN,2011-06-13
1,2,725,15,15,0,2011-06-03,2011-06-13,2011-06-14,NaN,2011-06-13
2,3,726,9,9,0,2011-06-03,2011-06-13,2011-06-14,NaN,2011-06-13
3,4,729,16,16,0,2011-06-03,2011-06-13,2011-06-14,NaN,2011-06-13
4,5,730,14,14,0,2011-06-03,2011-06-13,2011-06-14,NaN,2011-06-13


In [77]:
wo.groupby([pd.to_datetime(wo['StartDate']).dt.year, 'ProductID'])['WorkOrderID'].count().reset_index().head()

,StartDate,ProductID,WorkOrderID
0,2011,3,209
1,2011,316,209
2,2011,324,209
3,2011,327,209
4,2011,328,115


In [78]:
# 22 Query to find the total sales by month and by product category: Write a SQL query to find the total sales by month and by product category from the Sales SalesOrderHeader, Sales SalesOrderDetail, and Production Product tables 

sod = db_reader().read_table('Sales.SalesOrderDetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [79]:
soh = db_reader().read_table('Sales.SalesOrderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [80]:
sale = merge().merge_tbl(sod[['SalesOrderID','ProductID']], soh[['SalesOrderID', 'TotalDue', 'OrderDate']], 'inner', 'SalesOrderID')
sale.head()

,SalesOrderID,ProductID,TotalDue,OrderDate
0,43660,762,1457.3288,2011-05-20
1,43660,758,1457.3288,2011-05-20
2,43661,745,36865.8012,2011-05-31
3,43661,743,36865.8012,2011-05-31
4,43661,747,36865.8012,2011-05-31


In [81]:
p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [82]:
sc = merge().merge_tbl(sale, p[['ProductID', 'ProductSubcategoryID']], 'inner','ProductID')
sc.head()

,SalesOrderID,ProductID,TotalDue,OrderDate,ProductSubcategoryID
0,43660,762,1457.3288,2011-05-20,2.0
1,43660,758,1457.3288,2011-05-20,2.0
2,43661,745,36865.8012,2011-05-31,12.0
3,43661,743,36865.8012,2011-05-31,12.0
4,43661,747,36865.8012,2011-05-31,12.0


In [83]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [84]:
cat = merge().merge_tbl(sc, psc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner', 'ProductSubcategoryID')
cat.head()

,SalesOrderID,ProductID,TotalDue,OrderDate,ProductSubcategoryID,ProductCategoryID
0,43660,762,1457.3288,2011-05-20,2.0,1
1,43660,758,1457.3288,2011-05-20,2.0,1
2,43661,745,36865.8012,2011-05-31,12.0,2
3,43661,743,36865.8012,2011-05-31,12.0,2
4,43661,747,36865.8012,2011-05-31,12.0,2


In [85]:
cat.groupby([pd.to_datetime(cat['OrderDate']).dt.month_name(), 'ProductCategoryID'])['TotalDue'].sum().reset_index()

,OrderDate,ProductCategoryID,TotalDue
0,April,1,4.705512e+07
1,April,2,3.127466e+07
2,April,3,2.375017e+07
3,April,4,1.143410e+07
4,August,1,8.438786e+07
5,August,2,6.227659e+07
6,August,3,4.038820e+07
7,August,4,2.016269e+07
8,December,1,8.693038e+07
9,December,2,3.002981e+07


In [86]:
# 23 Query to find the total quantity of products sold by each sales territory: Write a SQL query to find the total quantity of products sold by each sales territory from the Sales SalesOrderHeader, Sales SalesOrderDetail, and Sales SalesTerritory tables 

sod = db_reader().read_table('Sales.SalesOrderDetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [87]:
soh = db_reader().read_table('Sales.SalesOrderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [88]:
sale = merge().merge_tbl(sod[['SalesOrderID','ProductID', 'OrderQty']], soh[['SalesOrderID', 'TerritoryID']], 'inner', 'SalesOrderID')
sale.head()

,SalesOrderID,ProductID,OrderQty,TerritoryID
0,43660,762,1,5
1,43660,758,1,5
2,43661,745,1,6
3,43661,743,1,6
4,43661,747,2,6


In [89]:
st = db_reader().read_table('Sales.SalesTerritory', connection)
st.head()

,TerritoryID,Name,CountryRegionCode,Group,SalesYTD,SalesLastYear,CostYTD,CostLastYear,rowguid,ModifiedDate
0,1,Northwest,US,North America,7.887187e+06,3.298694e+06,0.0,0.0,43689A10-E30B-497F-B0DE-11DE20267FF7,2008-04-30
1,2,Northeast,US,North America,2.402177e+06,3.607149e+06,0.0,0.0,00FB7309-96CC-49E2-8363-0A1BA72486F2,2008-04-30
2,3,Central,US,North America,3.072175e+06,3.205014e+06,0.0,0.0,DF6E7FD8-1A8D-468C-B103-ED8ADDB452C1,2008-04-30
3,4,Southwest,US,North America,1.051085e+07,5.366576e+06,0.0,0.0,DC3E9EA0-7950-4431-9428-99DBCBC33865,2008-04-30
4,5,Southeast,US,North America,2.538667e+06,3.925071e+06,0.0,0.0,6DC4165A-5E4C-42D2-809D-4344E0AC75E7,2008-04-30


In [90]:
t = merge().merge_tbl(sale, st['TerritoryID'], 'inner', 'TerritoryID')
t.head()

,SalesOrderID,ProductID,OrderQty,TerritoryID
0,43660,762,1,5
1,43660,758,1,5
2,43661,745,1,6
3,43661,743,1,6
4,43661,747,2,6


In [91]:
t.groupby('TerritoryID')['OrderQty'].sum().reset_index()

,TerritoryID,OrderQty
0,1,36776
1,2,19843
2,3,19493
3,4,59105
4,5,18849
5,6,49381
6,7,19906
7,8,13143
8,9,18293
9,10,20099


In [92]:
# 24 Query to find the total purchase order amount for each product by month: Write a SQL query to find the total purchase order amount for each product by month from the Purchasing PurchaseOrderHeader, Purchasing PurchaseOrderDetail, and Production Product tables 

poh = db_reader().read_table('Purchasing.PurchaseOrderHeader', connection)
poh.head()

,PurchaseOrderID,RevisionNumber,Status,EmployeeID,VendorID,ShipMethodID,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate
0,1,4,4,258,1580,3,2011-04-16,2011-04-25,201.0400,16.0832,5.0260,222.1492,2011-04-25
1,2,4,1,254,1496,5,2011-04-16,2011-04-25,272.1015,21.7681,6.8025,300.6721,2011-04-25
2,3,4,4,257,1494,2,2011-04-16,2011-04-25,8847.3000,707.7840,221.1825,9776.2665,2011-04-25
3,4,4,3,261,1650,5,2011-04-16,2011-04-25,171.0765,13.6861,4.2769,189.0395,2011-04-25
4,5,4,4,251,1654,4,2011-04-30,2011-05-09,20397.3000,1631.7840,509.9325,22539.0165,2011-05-09


In [93]:
pod = db_reader().read_table('Purchasing.PurchaseOrderDetail', connection)
pod.head()

,PurchaseOrderID,PurchaseOrderDetailID,DueDate,OrderQty,ProductID,UnitPrice,LineTotal,ReceivedQty,RejectedQty,StockedQty,ModifiedDate
0,1,1,2011-04-30,4,1,50.2600,201.0400,3.0,0.0,3.0,2011-04-23
1,2,2,2011-04-30,3,359,45.1200,135.3600,3.0,0.0,3.0,2011-04-23
2,2,3,2011-04-30,3,360,45.5805,136.7415,3.0,0.0,3.0,2011-04-23
3,3,4,2011-04-30,550,530,16.0860,8847.3000,550.0,0.0,550.0,2011-04-23
4,4,5,2011-04-30,3,4,57.0255,171.0765,2.0,1.0,1.0,2011-04-23


In [94]:
pur = merge().merge_tbl(poh[['PurchaseOrderID', 'OrderDate', 'TotalDue']], pod[['PurchaseOrderID', 'ProductID']],'inner' ,'PurchaseOrderID')
pur.head()

,PurchaseOrderID,OrderDate,TotalDue,ProductID
0,1,2011-04-16,222.1492,1
1,2,2011-04-16,300.6721,359
2,2,2011-04-16,300.6721,360
3,3,2011-04-16,9776.2665,530
4,4,2011-04-16,189.0395,4


In [95]:
p = db_reader().read_table('Production.Product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [96]:
prod = merge().merge_tbl(pur, p[['ProductID', 'Name']], 'inner', 'ProductID')
prod.head()

,PurchaseOrderID,OrderDate,TotalDue,ProductID,Name
0,1,2011-04-16,222.1492,1,Adjustable Race
1,2,2011-04-16,300.6721,359,Thin-Jam Hex Nut 9
2,2,2011-04-16,300.6721,360,Thin-Jam Hex Nut 10
3,3,2011-04-16,9776.2665,530,Seat Post
4,4,2011-04-16,189.0395,4,Headset Ball Bearings


In [97]:
prod.groupby([pd.to_datetime(prod['OrderDate']).dt.month_name(),'ProductID'])['TotalDue'].sum().reset_index().head()

,OrderDate,ProductID,TotalDue
0,April,1,1055.2667
1,April,2,694.7575
2,April,4,1134.2370
3,April,317,323146.4486
4,April,318,323146.4486


In [98]:
# 25 Query to find the total number of work orders for each product by month: Write a SQL query to find the total number of work orders for each product by month from the Production WorkOrder table 

wo = db_reader().read_table('Production.WorkOrder', connection)
wo.head()

,WorkOrderID,ProductID,OrderQty,StockedQty,ScrappedQty,StartDate,EndDate,DueDate,ScrapReasonID,ModifiedDate
0,1,722,8,8,0,2011-06-03,2011-06-13,2011-06-14,NaN,2011-06-13
1,2,725,15,15,0,2011-06-03,2011-06-13,2011-06-14,NaN,2011-06-13
2,3,726,9,9,0,2011-06-03,2011-06-13,2011-06-14,NaN,2011-06-13
3,4,729,16,16,0,2011-06-03,2011-06-13,2011-06-14,NaN,2011-06-13
4,5,730,14,14,0,2011-06-03,2011-06-13,2011-06-14,NaN,2011-06-13


In [99]:
wo.groupby([pd.to_datetime(wo['StartDate']).dt.month_name(), 'ProductID'])['WorkOrderID'].count().reset_index().head()

,StartDate,ProductID,WorkOrderID
0,April,3,90
1,April,316,90
2,April,324,90
3,April,327,90
4,April,328,84


In [100]:
# 26 Query to find the total sales for each product category by sales territory: Write a SQL query to find the total sales for each product category by sales territory from the Sales SalesOrderHeader, Sales SalesOrderDetail, Sales SalesTerritory, and Production Product tables

sod = db_reader().read_table('Sales.SalesOrderDetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [101]:
soh = db_reader().read_table('Sales.SalesOrderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [102]:
sale = merge().merge_tbl(sod[['SalesOrderID','ProductID']], soh[['SalesOrderID', 'TerritoryID', 'TotalDue']], 'inner', 'SalesOrderID')
sale.head()

,SalesOrderID,ProductID,TerritoryID,TotalDue
0,43660,762,5,1457.3288
1,43660,758,5,1457.3288
2,43661,745,6,36865.8012
3,43661,743,6,36865.8012
4,43661,747,6,36865.8012


In [103]:
st = db_reader().read_table('Sales.SalesTerritory', connection)
st.head()

,TerritoryID,Name,CountryRegionCode,Group,SalesYTD,SalesLastYear,CostYTD,CostLastYear,rowguid,ModifiedDate
0,1,Northwest,US,North America,7.887187e+06,3.298694e+06,0.0,0.0,43689A10-E30B-497F-B0DE-11DE20267FF7,2008-04-30
1,2,Northeast,US,North America,2.402177e+06,3.607149e+06,0.0,0.0,00FB7309-96CC-49E2-8363-0A1BA72486F2,2008-04-30
2,3,Central,US,North America,3.072175e+06,3.205014e+06,0.0,0.0,DF6E7FD8-1A8D-468C-B103-ED8ADDB452C1,2008-04-30
3,4,Southwest,US,North America,1.051085e+07,5.366576e+06,0.0,0.0,DC3E9EA0-7950-4431-9428-99DBCBC33865,2008-04-30
4,5,Southeast,US,North America,2.538667e+06,3.925071e+06,0.0,0.0,6DC4165A-5E4C-42D2-809D-4344E0AC75E7,2008-04-30


In [104]:
t = merge().merge_tbl(sale, st['TerritoryID'], 'inner', 'TerritoryID')
t.head()

,SalesOrderID,ProductID,TerritoryID,TotalDue
0,43660,762,5,1457.3288
1,43660,758,5,1457.3288
2,43661,745,6,36865.8012
3,43661,743,6,36865.8012
4,43661,747,6,36865.8012


In [105]:
pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [106]:
sc = merge().merge_tbl(t, pro[['ProductID', 'ProductSubcategoryID']], 'inner', 'ProductID')
sc.head()

,SalesOrderID,ProductID,TerritoryID,TotalDue,ProductSubcategoryID
0,43660,762,5,1457.3288,2.0
1,43660,758,5,1457.3288,2.0
2,43661,745,6,36865.8012,12.0
3,43661,743,6,36865.8012,12.0
4,43661,747,6,36865.8012,12.0


In [107]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [108]:
cat = merge().merge_tbl(sc, psc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner', 'ProductSubcategoryID')
cat.head()

,SalesOrderID,ProductID,TerritoryID,TotalDue,ProductSubcategoryID,ProductCategoryID
0,43660,762,5,1457.3288,2.0,1
1,43660,758,5,1457.3288,2.0,1
2,43661,745,6,36865.8012,12.0,2
3,43661,743,6,36865.8012,12.0,2
4,43661,747,6,36865.8012,12.0,2


In [109]:
cat.groupby(['TerritoryID', 'ProductCategoryID'])['TotalDue'].sum().reset_index().head()

,TerritoryID,ProductCategoryID,TotalDue
0,1,1,1.716123e+08
1,1,2,1.413131e+08
2,1,3,6.718148e+07
3,1,4,3.110041e+07
4,2,1,1.026269e+08


In [110]:
# 27 Query to find the total sales by each salesperson: Write a SQL query to find the total sales by each salesperson from the Sales SalesOrderHeader and Sales SalesPerson tables

soh = db_reader().read_table('Sales.SalesOrderHeader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [111]:
sp = db_reader().read_table('Sales.SalesPerson', connection)
sp.head()

,BusinessEntityID,TerritoryID,SalesQuota,Bonus,CommissionPct,SalesYTD,SalesLastYear,rowguid,ModifiedDate
0,274,NaN,NaN,0.0,0.000,5.596976e+05,0.000000e+00,48754992-9EE0-4C0E-8C94-9451604E3E02,2010-12-28
1,275,2.0,300000.0,4100.0,0.012,3.763178e+06,1.750406e+06,1E0A7274-3064-4F58-88EE-4C6586C87169,2011-05-24
2,276,4.0,250000.0,2000.0,0.015,4.251369e+06,1.439156e+06,4DD9EEE4-8E81-4F8C-AF97-683394C1F7C0,2011-05-24
3,277,3.0,250000.0,2500.0,0.015,3.189418e+06,1.997186e+06,39012928-BFEC-4242-874D-423162C3F567,2011-05-24
4,278,6.0,250000.0,500.0,0.010,1.453719e+06,1.620277e+06,7A0AE1AB-B283-40F9-91D1-167ABF06D720,2011-05-24


In [112]:
p = merge().merge_tbl(soh[['SalesPersonID', 'TotalDue']], sp['BusinessEntityID'], 'inner', col1 = 'SalesPersonID', col2 = 'BusinessEntityID')
p.head()

,SalesPersonID,TotalDue,BusinessEntityID
0,279.0,1457.3288,279
1,282.0,36865.8012,282
2,282.0,32474.9324,282
3,276.0,472.3108,276
4,280.0,27510.4109,280


In [113]:
p.groupby('BusinessEntityID')['TotalDue'].sum().reset_index().head()

,BusinessEntityID,TotalDue
0,274,1.235934e+06
1,275,1.047537e+07
2,276,1.169502e+07
3,277,1.134239e+07
4,278,4.069422e+06


In [114]:
# 28 Query to find the total quantity of products purchased by each vendor: Write a SQL query to find the total quantity of products purchased by each vendor from the Purchasing PurchaseOrderHeader, Purchasing PurchaseOrderDetail, and Purchasing Vendor tables

poh = db_reader().read_table('Purchasing.PurchaseOrderHeader', connection)
poh.head()

,PurchaseOrderID,RevisionNumber,Status,EmployeeID,VendorID,ShipMethodID,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate
0,1,4,4,258,1580,3,2011-04-16,2011-04-25,201.0400,16.0832,5.0260,222.1492,2011-04-25
1,2,4,1,254,1496,5,2011-04-16,2011-04-25,272.1015,21.7681,6.8025,300.6721,2011-04-25
2,3,4,4,257,1494,2,2011-04-16,2011-04-25,8847.3000,707.7840,221.1825,9776.2665,2011-04-25
3,4,4,3,261,1650,5,2011-04-16,2011-04-25,171.0765,13.6861,4.2769,189.0395,2011-04-25
4,5,4,4,251,1654,4,2011-04-30,2011-05-09,20397.3000,1631.7840,509.9325,22539.0165,2011-05-09


In [115]:
pod = db_reader().read_table('Purchasing.PurchaseOrderDetail', connection)
pod.head()

,PurchaseOrderID,PurchaseOrderDetailID,DueDate,OrderQty,ProductID,UnitPrice,LineTotal,ReceivedQty,RejectedQty,StockedQty,ModifiedDate
0,1,1,2011-04-30,4,1,50.2600,201.0400,3.0,0.0,3.0,2011-04-23
1,2,2,2011-04-30,3,359,45.1200,135.3600,3.0,0.0,3.0,2011-04-23
2,2,3,2011-04-30,3,360,45.5805,136.7415,3.0,0.0,3.0,2011-04-23
3,3,4,2011-04-30,550,530,16.0860,8847.3000,550.0,0.0,550.0,2011-04-23
4,4,5,2011-04-30,3,4,57.0255,171.0765,2.0,1.0,1.0,2011-04-23


In [116]:
pur = merge().merge_tbl(poh[['PurchaseOrderID', 'VendorID']], pod[['PurchaseOrderID', 'OrderQty']], 'inner', 'PurchaseOrderID')
pur.head()

,PurchaseOrderID,VendorID,OrderQty
0,1,1580,4
1,2,1496,3
2,2,1496,3
3,3,1494,550
4,4,1650,3


In [117]:
pv = db_reader().read_table('Purchasing.Vendor', connection)
pv.head()

,BusinessEntityID,AccountNumber,Name,CreditRating,PreferredVendorStatus,ActiveFlag,PurchasingWebServiceURL,ModifiedDate
0,1492,AUSTRALI0001,Australia Bike Retailer,1,True,True,None,2011-12-23
1,1494,ALLENSON0001,Allenson Cycles,2,True,True,None,2011-04-25
2,1496,ADVANCED0001,Advanced Bicycles,1,True,True,None,2011-04-25
3,1498,TRIKES0001,"Trikes, Inc.",2,True,True,None,2012-02-03
4,1500,MORGANB0001,Morgan Bike Accessories,1,True,True,None,2012-02-02


In [118]:
v = merge().merge_tbl(pur, pv[['BusinessEntityID', 'Name']], 'inner', col1 = 'VendorID', col2 = 'BusinessEntityID')
v.head()

,PurchaseOrderID,VendorID,OrderQty,BusinessEntityID,Name
0,1,1580,4,1580,"Litware, Inc."
1,2,1496,3,1496,Advanced Bicycles
2,2,1496,3,1496,Advanced Bicycles
3,3,1494,550,1494,Allenson Cycles
4,4,1650,3,1650,American Bicycles and Wheels


In [119]:
v.groupby(['BusinessEntityID', 'Name'])['OrderQty'].count().reset_index().head()

,BusinessEntityID,Name,OrderQty
0,1492,Australia Bike Retailer,176
1,1494,Allenson Cycles,51
2,1496,Advanced Bicycles,199
3,1498,"Trikes, Inc.",100
4,1500,Morgan Bike Accessories,50


In [120]:
# 29 Query to find the total number of work orders for each product by year: Write a SQL query to find the total number of work orders for each product by year from the Production WorkOrder table 

wo = db_reader().read_table('Production.WorkOrder', connection)
wo.head()

,WorkOrderID,ProductID,OrderQty,StockedQty,ScrappedQty,StartDate,EndDate,DueDate,ScrapReasonID,ModifiedDate
0,1,722,8,8,0,2011-06-03,2011-06-13,2011-06-14,NaN,2011-06-13
1,2,725,15,15,0,2011-06-03,2011-06-13,2011-06-14,NaN,2011-06-13
2,3,726,9,9,0,2011-06-03,2011-06-13,2011-06-14,NaN,2011-06-13
3,4,729,16,16,0,2011-06-03,2011-06-13,2011-06-14,NaN,2011-06-13
4,5,730,14,14,0,2011-06-03,2011-06-13,2011-06-14,NaN,2011-06-13


In [121]:
wo.groupby([pd.to_datetime(wo['StartDate']).dt.year,'ProductID'])['WorkOrderID'].count().reset_index().head()

,StartDate,ProductID,WorkOrderID
0,2011,3,209
1,2011,316,209
2,2011,324,209
3,2011,327,209
4,2011,328,115


In [122]:
# 30 Query to find the total sales for each product subcategory by sales territory: Write a SQL query to find the total sales for each product subcategory by sales territory from the Sales SalesOrderHeader, Sales SalesOrderDetail, Sales SalesTerritory, and Production Product tables

sod = db_reader().read_table('Sales.SalesOrderDetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [123]:
soh = db_reader().read_table('Sales.SalesOrderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [124]:
sale = merge().merge_tbl(sod[['SalesOrderID','ProductID', 'OrderQty']], soh[['SalesOrderID', 'TerritoryID', 'TotalDue']], 'inner', 'SalesOrderID')
sale.head()

,SalesOrderID,ProductID,OrderQty,TerritoryID,TotalDue
0,43660,762,1,5,1457.3288
1,43660,758,1,5,1457.3288
2,43661,745,1,6,36865.8012
3,43661,743,1,6,36865.8012
4,43661,747,2,6,36865.8012


In [125]:
st = db_reader().read_table('Sales.SalesTerritory', connection)
st.head()

,TerritoryID,Name,CountryRegionCode,Group,SalesYTD,SalesLastYear,CostYTD,CostLastYear,rowguid,ModifiedDate
0,1,Northwest,US,North America,7.887187e+06,3.298694e+06,0.0,0.0,43689A10-E30B-497F-B0DE-11DE20267FF7,2008-04-30
1,2,Northeast,US,North America,2.402177e+06,3.607149e+06,0.0,0.0,00FB7309-96CC-49E2-8363-0A1BA72486F2,2008-04-30
2,3,Central,US,North America,3.072175e+06,3.205014e+06,0.0,0.0,DF6E7FD8-1A8D-468C-B103-ED8ADDB452C1,2008-04-30
3,4,Southwest,US,North America,1.051085e+07,5.366576e+06,0.0,0.0,DC3E9EA0-7950-4431-9428-99DBCBC33865,2008-04-30
4,5,Southeast,US,North America,2.538667e+06,3.925071e+06,0.0,0.0,6DC4165A-5E4C-42D2-809D-4344E0AC75E7,2008-04-30


In [126]:
t = merge().merge_tbl(sale, st['TerritoryID'], 'inner', 'TerritoryID')
t.head()

,SalesOrderID,ProductID,OrderQty,TerritoryID,TotalDue
0,43660,762,1,5,1457.3288
1,43660,758,1,5,1457.3288
2,43661,745,1,6,36865.8012
3,43661,743,1,6,36865.8012
4,43661,747,2,6,36865.8012


In [127]:
pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [128]:
sc = merge().merge_tbl(t, pro[['ProductID', 'ProductSubcategoryID']], 'inner', 'ProductID')
sc.head()

,SalesOrderID,ProductID,OrderQty,TerritoryID,TotalDue,ProductSubcategoryID
0,43660,762,1,5,1457.3288,2.0
1,43660,758,1,5,1457.3288,2.0
2,43661,745,1,6,36865.8012,12.0
3,43661,743,1,6,36865.8012,12.0
4,43661,747,2,6,36865.8012,12.0


In [129]:
sc.groupby(['TerritoryID', 'ProductSubcategoryID'])['TotalDue'].sum().reset_index().head()

,TerritoryID,ProductSubcategoryID,TotalDue
0,1,1.0,7.751736e+07
1,1,2.0,5.729189e+07
2,1,3.0,3.680301e+07
3,1,4.0,1.248118e+07
4,1,5.0,3.665540e+06


In [130]:
#31 Query to find the total purchase order amount for each product by vendor: Write a SQL query to find the total purchase order amount for each product by vendor from the Purchasing PurchaseOrderHeader, Purchasing PurchaseOrderDetail, Purchasing Vendor, and Production Product tables

poh = db_reader().read_table('Purchasing.PurchaseOrderHeader', connection)
poh.head()

,PurchaseOrderID,RevisionNumber,Status,EmployeeID,VendorID,ShipMethodID,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate
0,1,4,4,258,1580,3,2011-04-16,2011-04-25,201.0400,16.0832,5.0260,222.1492,2011-04-25
1,2,4,1,254,1496,5,2011-04-16,2011-04-25,272.1015,21.7681,6.8025,300.6721,2011-04-25
2,3,4,4,257,1494,2,2011-04-16,2011-04-25,8847.3000,707.7840,221.1825,9776.2665,2011-04-25
3,4,4,3,261,1650,5,2011-04-16,2011-04-25,171.0765,13.6861,4.2769,189.0395,2011-04-25
4,5,4,4,251,1654,4,2011-04-30,2011-05-09,20397.3000,1631.7840,509.9325,22539.0165,2011-05-09


In [131]:
pod = db_reader().read_table('Purchasing.PurchaseOrderDetail', connection)
pod.head()

,PurchaseOrderID,PurchaseOrderDetailID,DueDate,OrderQty,ProductID,UnitPrice,LineTotal,ReceivedQty,RejectedQty,StockedQty,ModifiedDate
0,1,1,2011-04-30,4,1,50.2600,201.0400,3.0,0.0,3.0,2011-04-23
1,2,2,2011-04-30,3,359,45.1200,135.3600,3.0,0.0,3.0,2011-04-23
2,2,3,2011-04-30,3,360,45.5805,136.7415,3.0,0.0,3.0,2011-04-23
3,3,4,2011-04-30,550,530,16.0860,8847.3000,550.0,0.0,550.0,2011-04-23
4,4,5,2011-04-30,3,4,57.0255,171.0765,2.0,1.0,1.0,2011-04-23


In [132]:
pur = merge().merge_tbl(poh[['PurchaseOrderID', 'VendorID', 'TotalDue']], pod[['PurchaseOrderID', 'ProductID']], 'inner', 'PurchaseOrderID')
pur.head()

,PurchaseOrderID,VendorID,TotalDue,ProductID
0,1,1580,222.1492,1
1,2,1496,300.6721,359
2,2,1496,300.6721,360
3,3,1494,9776.2665,530
4,4,1650,189.0395,4


In [133]:
pv = db_reader().read_table('Purchasing.Vendor', connection)
pv.head()

,BusinessEntityID,AccountNumber,Name,CreditRating,PreferredVendorStatus,ActiveFlag,PurchasingWebServiceURL,ModifiedDate
0,1492,AUSTRALI0001,Australia Bike Retailer,1,True,True,None,2011-12-23
1,1494,ALLENSON0001,Allenson Cycles,2,True,True,None,2011-04-25
2,1496,ADVANCED0001,Advanced Bicycles,1,True,True,None,2011-04-25
3,1498,TRIKES0001,"Trikes, Inc.",2,True,True,None,2012-02-03
4,1500,MORGANB0001,Morgan Bike Accessories,1,True,True,None,2012-02-02


In [134]:
v = merge().merge_tbl(pur, pv[['BusinessEntityID']], 'inner', col1 = 'VendorID', col2 = 'BusinessEntityID')
v.head()

,PurchaseOrderID,VendorID,TotalDue,ProductID,BusinessEntityID
0,1,1580,222.1492,1,1580
1,2,1496,300.6721,359,1496
2,2,1496,300.6721,360,1496
3,3,1494,9776.2665,530,1494
4,4,1650,189.0395,4,1650


In [135]:
p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [136]:
prod = merge().merge_tbl(v, p[['ProductID', 'Name']], 'inner', 'ProductID')
prod.head()

,PurchaseOrderID,VendorID,TotalDue,ProductID,BusinessEntityID,Name
0,1,1580,222.1492,1,1580,Adjustable Race
1,2,1496,300.6721,359,1496,Thin-Jam Hex Nut 9
2,2,1496,300.6721,360,1496,Thin-Jam Hex Nut 10
3,3,1494,9776.2665,530,1494,Seat Post
4,4,1650,189.0395,4,1650,Headset Ball Bearings


In [137]:
prod.groupby(['BusinessEntityID', 'ProductID','Name'])['TotalDue'].sum().reset_index().head()

,BusinessEntityID,ProductID,Name,TotalDue
0,1492,422,Thin-Jam Lock Nut 9,5999.2811
1,1492,423,Thin-Jam Lock Nut 10,5999.2811
2,1492,424,Thin-Jam Lock Nut 1,6609.7699
3,1492,425,Thin-Jam Lock Nut 2,6609.7699
4,1492,426,Thin-Jam Lock Nut 15,7770.7741


In [138]:
# 32 Retrieve the current system date and time.

pd.Timestamp.now()

Timestamp('2024-06-20 17:09:30.463928')

In [139]:
# 33 List all orders placed after January 1, 2023.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [140]:
soh[soh['OrderDate'] > 'January 1, 2023'].head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate


In [141]:
# 34 Find the number of days between the order date and the ship date for each order

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [142]:
soh['Diff_Of_Days'] = pd.to_datetime(soh['ShipDate']) - pd.to_datetime(soh['OrderDate'])

In [143]:
soh[['SalesOrderID', 'OrderDate', 'ShipDate', 'Diff_Of_Days']].head()

,SalesOrderID,OrderDate,ShipDate,Diff_Of_Days
0,43660,2011-05-20,2011-06-07,18 days
1,43661,2011-05-31,2011-06-07,7 days
2,43662,2011-05-31,2011-06-07,7 days
3,43663,2011-05-31,2011-06-07,7 days
4,43664,2011-05-31,2011-06-07,7 days


In [144]:
# 35 Retrieve the total number of orders placed in each month of the year 2023.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [145]:
soh[pd.to_datetime(soh['OrderDate']).dt.year == 2023]

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate


In [146]:
# 36 List all orders placed on weekends (Saturday or Sunday).

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [147]:
soh[pd.to_datetime(soh['OrderDate']).dt.weekday >= 5].head() #mon = 0, ....sun = 6

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
53,43713,8,2011-06-04,2011-06-16,2011-06-11,5,True,SO43713,None,10-4030-027601,27601,NaN,4,11855,11855,1,16570.0,830288Vi85808,NaN,3578.2700,286.2616,89.4568,3953.9884,None,9DE30294-9066-4988-A3AD-09A0713348E5,2011-06-11
54,43714,8,2011-06-04,2011-06-16,2011-06-11,5,True,SO43714,None,10-4030-013591,13591,NaN,10,15169,15169,1,15963.0,1031124Vi82522,61.0,3578.2700,286.2616,89.4568,3953.9884,None,40659773-9CE3-4214-9886-C9DE75E40274,2011-06-11
55,43715,8,2011-06-04,2011-06-16,2011-06-11,5,True,SO43715,None,10-4030-016483,16483,NaN,9,28213,28213,1,7985.0,1134605Vi41375,54.0,3578.2700,286.2616,89.4568,3953.9884,None,36C19607-CC19-4CC0-8EA7-C16A6CC747DD,2011-06-11
56,43716,8,2011-06-04,2011-06-16,2011-06-11,5,True,SO43716,None,10-4030-016529,16529,NaN,9,23557,23557,1,5873.0,534626Vi30635,54.0,3578.2700,286.2616,89.4568,3953.9884,None,92D7B8BF-071B-478E-A330-DEC813C635F9,2011-06-11
57,43717,8,2011-06-04,2011-06-16,2011-06-11,5,True,SO43717,None,10-4030-025249,25249,NaN,9,11809,11809,1,5700.0,1235425Vi29858,54.0,699.0982,55.9279,17.4775,772.5036,None,8CF612EF-31B8-4A57-8E57-5C62D48A0944,2011-06-11


In [148]:
# 37 Find the average number of days it took to ship orders for each customer

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [149]:
soh['avg_Diff_Of_Days'] = (soh['ShipDate'] - soh['OrderDate']).dt.days.mean()

In [150]:
soh[['SalesOrderID', 'OrderDate', 'ShipDate', 'avg_Diff_Of_Days']].head()

,SalesOrderID,OrderDate,ShipDate,avg_Diff_Of_Days
0,43660,2011-05-20,2011-06-07,7.000636
1,43661,2011-05-31,2011-06-07,7.000636
2,43662,2011-05-31,2011-06-07,7.000636
3,43663,2011-05-31,2011-06-07,7.000636
4,43664,2011-05-31,2011-06-07,7.000636


In [151]:
# 38 Retrieve all orders placed in the last quarter of 2023.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [152]:

soh[(soh['OrderDate'].dt.year == 2013) & (soh['OrderDate'].dt.quarter == 4)].tail()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
19698,63358,8,2013-12-31,2014-01-12,2014-01-07,5,True,SO63358,None,10-4030-020826,20826,NaN,7,24387,24387,1,3239.0,1142084Vi17039,NaN,1173.96,93.9168,29.3490,1297.2258,None,41278FBB-3DD8-488B-AEA5-8BF4A6F1C52A,2014-01-07
19699,63359,8,2013-12-31,2014-01-12,2014-01-07,5,True,SO63359,None,10-4030-024114,24114,NaN,10,29682,29682,1,NaN,None,10770.0,1179.47,94.3576,29.4868,1303.3144,None,749532D4-BFF7-4FEC-9F77-396C4A96AD2F,2014-01-07
19700,63360,8,2013-12-31,2014-01-12,2014-01-07,5,True,SO63360,None,10-4030-011417,11417,NaN,7,21465,21465,1,6582.0,242387Vi34223,NaN,548.98,43.9184,13.7245,606.6229,None,F92FA2A3-73E1-4DD0-987C-99D1C87EA68C,2014-01-07
19701,63361,8,2013-12-31,2014-01-12,2014-01-07,5,True,SO63361,None,10-4030-018125,18125,NaN,8,26562,26562,1,11994.0,1242859Vi61993,NaN,2384.07,190.7256,59.6018,2634.3974,None,3A3758BF-CDFA-4740-9104-87BD5A084BDB,2014-01-07
19702,63362,8,2013-12-31,2014-01-12,2014-01-07,5,True,SO63362,None,10-4030-015692,15692,NaN,10,27090,27090,1,16083.0,242864Vi83167,10770.0,2419.06,193.5248,60.4765,2673.0613,None,3151D31F-4020-41B2-87F3-B6732869EF38,2014-01-07


In [153]:
# 39 List all orders placed on weekdays (Monday to Friday) between 9:00 AM and 5:00 PM.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [154]:
soh[(soh['OrderDate'].dt.weekday >= 0) & (soh['OrderDate'].dt.weekday <= 4) & (soh['OrderDate'].dt.hour >= 9) & (soh['OrderDate'].dt.hour <= 17)].head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate


In [155]:
# 40 Find the total sales amount for each month of the year 2023.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [156]:
soh = soh[soh['OrderDate'].dt.year == 2013]

In [157]:
soh.groupby([(soh['OrderDate'].dt.year).rename('Year'), (soh['OrderDate'].dt.month_name().rename('Month'))])['TotalDue'].sum().reset_index().head()

,Year,Month,TotalDue
0,2013,April,2.840711e+06
1,2013,August,3.733973e+06
2,2013,December,4.560577e+06
3,2013,February,2.600219e+06
4,2013,January,2.340062e+06


In [158]:
# 41 Retrieve the orders placed on the same day they were shipped.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [159]:
soh[(soh['ShipDate'] - soh['OrderDate']).dt.days == 0].head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate


In [160]:
# 42 List all orders placed on Mondays.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [161]:
soh[soh['OrderDate'].dt.weekday == 0].head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
62,43722,8,2011-06-06,2011-06-18,2011-06-13,5,True,SO43722,None,10-4030-013581,13581,NaN,10,23990,23990,1,12191.0,331099Vi63048,87.0,3578.2700,286.2616,89.4568,3953.9884,None,681FEC9E-C1FB-42C9-A96C-48C911B086DD,2011-06-13
63,43723,8,2011-06-06,2011-06-18,2011-06-13,5,True,SO43723,None,10-4030-014520,14520,NaN,1,12708,12708,1,16949.0,833793Vi87871,NaN,699.0982,55.9279,17.4775,772.5036,None,4EB5EC63-D247-468F-B88B-C2E5C019D949,2011-06-13
64,43724,8,2011-06-06,2011-06-18,2011-06-13,5,True,SO43724,None,10-4030-016520,16520,NaN,9,25572,25572,1,6336.0,834617Vi32891,80.0,3578.2700,286.2616,89.4568,3953.9884,None,62FCAB48-3494-4AF8-A1CB-5E2FE0929FF0,2011-06-13
87,43747,8,2011-06-13,2011-06-25,2011-06-20,5,True,SO43747,None,10-4030-027623,27623,NaN,4,27241,27241,1,16044.0,1230190Vi82952,NaN,3578.2700,286.2616,89.4568,3953.9884,None,5DC249A1-592C-472E-93B7-E2F26DDE5DAB,2011-06-20
88,43748,8,2011-06-13,2011-06-25,2011-06-20,5,True,SO43748,None,10-4030-027625,27625,NaN,1,16663,16663,1,14948.0,430191Vi77442,NaN,3578.2700,286.2616,89.4568,3953.9884,None,5D25E2A5-A4C6-45AA-B388-9BD83769A2AF,2011-06-20


In [162]:
# 43 Find the average time taken to ship orders, excluding weekends

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [163]:
soh['avg_Diff_Of_Days'] = (soh['ShipDate'] - soh['OrderDate']).dt.days.mean()

In [164]:
soh.loc[(soh['ShipDate'].dt.weekday >= 0) & (soh['ShipDate'].dt.weekday <= 4), ['SalesOrderID', 'ShipDate', 'avg_Diff_Of_Days']].head()

,SalesOrderID,ShipDate,avg_Diff_Of_Days
0,43660,2011-06-07,7.000636
1,43661,2011-06-07,7.000636
2,43662,2011-06-07,7.000636
3,43663,2011-06-07,7.000636
4,43664,2011-06-07,7.000636


In [165]:
# 44 Retrieve the number of orders placed in each quarter of the year 2023.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [166]:
soh = soh[soh['OrderDate'].dt.year == 2013]

In [167]:
soh.groupby(soh['OrderDate'].dt.quarter.rename('quarter'))['SalesOrderID'].count().rename('count').reset_index().head()

,quarter,count
0,1,1166
1,2,1575
2,3,5320
3,4,6121


In [168]:
# 45 List all orders placed on the last day of each month.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [169]:
month_end = soh['OrderDate'] + pd.offsets.MonthEnd()
month_end

0       2011-05-31
1       2011-06-30
2       2011-06-30
3       2011-06-30
4       2011-06-30
5       2011-06-30
6       2011-06-30
7       2011-06-30
8       2011-06-30
9       2011-06-30
10      2011-06-30
11      2011-06-30
12      2011-06-30
13      2011-06-30
14      2011-06-30
15      2011-06-30
16      2011-06-30
17      2011-06-30
18      2011-06-30
19      2011-06-30
20      2011-06-30
21      2011-06-30
22      2011-06-30
23      2011-06-30
24      2011-06-30
25      2011-06-30
26      2011-06-30
27      2011-06-30
28      2011-06-30
29      2011-06-30
30      2011-06-30
31      2011-06-30
32      2011-06-30
33      2011-06-30
34      2011-06-30
35      2011-06-30
36      2011-06-30
37      2011-06-30
38      2011-06-30
39      2011-06-30
40      2011-06-30
41      2011-06-30
42      2011-06-30
43      2011-06-30
44      2011-06-30
45      2011-06-30
46      2011-06-30
47      2011-06-30
48      2011-06-30
49      2011-06-30
50      2011-06-30
51      2011-06-30
52      2011

In [170]:
soh[soh['OrderDate'].isin(month_end)].tail()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
31460,75120,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75120,None,10-4030-018749,18749,NaN,6,28374,28374,1,8925.0,929849Vi46003,NaN,84.9600,6.7968,2.1240,93.8808,None,AE6A4FCF-FF73-4CD4-AF2C-5993D00D4AFE,2014-07-07
31461,75121,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75121,None,10-4030-015251,15251,NaN,6,26553,26553,1,14220.0,529864Vi73738,NaN,74.9800,5.9984,1.8745,82.8529,None,D7395C0E-00CB-4BFA-A238-0D6A9F49884F,2014-07-07
31462,75122,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75122,None,10-4030-015868,15868,NaN,6,14616,14616,1,18719.0,330022Vi97312,NaN,30.9700,2.4776,0.7743,34.2219,None,4221035A-4159-492F-AF40-4363A64FFC16,2014-07-07
31463,75123,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75123,None,10-4030-018759,18759,NaN,6,14024,14024,1,10084.0,230370Vi51970,NaN,189.9700,15.1976,4.7493,209.9169,None,D54752FF-2B54-4BE5-95EA-3B72289C059F,2014-07-07
31464,75124,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO75124,PO18009186470,-4656,29614,283.0,5,517,849,5,15232.0,55680Vi53503,4.0,20565.6206,1971.5149,616.0984,23153.2339,None,0F91DBCB-04AD-4D4D-AEED-C2AF783D81B9,2011-06-07


In [171]:
# 46 Retrieve the orders placed exactly one year ago from the current date.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [172]:
dat = pd.Timestamp.now()-pd.DateOffset(years = 1)
dat

Timestamp('2023-06-20 17:09:57.526665')

In [173]:
soh[soh['OrderDate'].isin([dat])]

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate


In [174]:
# 47 Find the total number of orders placed in each hour of the day.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [175]:
soh.groupby([soh['OrderDate'].dt.hour.rename('Hour'), 'OrderDate'])['SalesOrderID'].count().reset_index().head()

,Hour,OrderDate,SalesOrderID
0,0,2011-05-20,1
1,0,2011-05-31,42
2,0,2011-06-01,4
3,0,2011-06-02,5
4,0,2011-06-03,2


In [176]:
# 48 Retrieve the orders placed in the month of December for each year

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [177]:
soh[soh['OrderDate'].dt.month_name() == 'December'].head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
1378,45038,8,2011-12-01,2011-12-13,2011-12-08,5,False,SO45038,PO18473129920,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85287Vi6854,2396.0,27123.6534,2613.3362,816.6676,30553.6572,None,2C419B24-B1F6-4D9D-B2BF-1E68389B473B,2011-12-08
1379,45039,8,2011-12-01,2011-12-13,2011-12-08,5,False,SO45039,PO18444197500,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125291Vi53935,2396.0,27722.1591,2666.1894,833.1842,31221.5327,None,0AF59811-C914-490B-ABF9-9984710296EB,2011-12-08
1380,45040,8,2011-12-01,2011-12-13,2011-12-08,5,False,SO45040,PO18009115235,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45356Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,636DF5F9-B1F1-47D9-8747-3599B00C1F5D,2011-12-08
1381,45041,8,2011-12-01,2011-12-13,2011-12-08,5,False,SO45041,PO17458147053,10-4020-000577,29858,280.0,1,894,894,5,6646.0,75510Vi34603,NaN,419.4589,40.2681,12.5838,472.3108,None,311D3ADF-9B54-4E38-8314-0E776E42AA5F,2011-12-08
1382,45042,8,2011-12-01,2011-12-13,2011-12-08,5,False,SO45042,PO16617111461,10-4020-000397,29898,280.0,1,876,876,5,806.0,95514Vi4081,NaN,40648.3800,3901.9785,1219.3683,45769.7268,None,60ADC018-AC87-4919-8949-8312A9319C7C,2011-12-08


In [178]:
# 49 Find the number of days elapsed between the order date and today's date for each order.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [179]:
soh['elapsed'] = (pd.Timestamp.now()- soh['OrderDate']).dt.days

In [180]:
soh[['OrderDate', 'elapsed']].head()

,OrderDate,elapsed
0,2011-05-20,4780
1,2011-05-31,4769
2,2011-05-31,4769
3,2011-05-31,4769
4,2011-05-31,4769


In [181]:
# 50 List all orders placed during the first half of each month.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [182]:
soh[soh['OrderDate'].dt.day <=15].head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
42,43702,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43702,None,10-4030-027645,27645,NaN,4,22584,22584,1,8087.0,1230194Vi41919,NaN,3578.27,286.2616,89.4568,3953.9884,None,9310C7F0-9A84-4CE9-BE08-F700FB1AADF7,2011-06-08
43,43703,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43703,None,10-4030-016624,16624,NaN,9,23967,23967,1,9497.0,1234632Vi48985,15.0,3578.27,286.2616,89.4568,3953.9884,None,CC630723-57F1-4E59-B1EF-BF77930B3906,2011-06-08
44,43704,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43704,None,10-4030-011005,11005,NaN,9,16025,16025,1,1988.0,835095Vi10342,15.0,3374.99,269.9992,84.3748,3729.3640,None,B09AF2D1-243D-444F-9784-FCFCEB3FBB0A,2011-06-08
45,43705,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43705,None,10-4030-011011,11011,NaN,9,27345,27345,1,7205.0,1035101Vi37381,15.0,3399.99,271.9992,84.9998,3756.9890,None,17A0EDE6-6F1A-419D-BCF9-3638B79166D0,2011-06-08
46,43706,8,2011-06-02,2011-06-14,2011-06-09,5,True,SO43706,None,10-4030-027621,27621,NaN,4,13264,13264,1,10912.0,230189Vi56258,NaN,3578.27,286.2616,89.4568,3953.9884,None,F02C4CB6-A5B5-4CE4-9473-CB084E383196,2011-06-09


In [183]:
# 51 Retrieve the orders placed in the first and last weeks of each month.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [184]:
soh[(soh['OrderDate'].dt.dayofweek == 0) | (soh['OrderDate'].dt.dayofweek == 3)].tail()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
31459,75119,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75119,None,10-4030-011981,11981,NaN,1,17649,17649,1,6761.0,429826Vi35166,NaN,42.28,3.3824,1.0570,46.7194,None,9382F1C9-383A-435F-9449-0EECEA21B78D,2014-07-07
31460,75120,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75120,None,10-4030-018749,18749,NaN,6,28374,28374,1,8925.0,929849Vi46003,NaN,84.96,6.7968,2.1240,93.8808,None,AE6A4FCF-FF73-4CD4-AF2C-5993D00D4AFE,2014-07-07
31461,75121,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75121,None,10-4030-015251,15251,NaN,6,26553,26553,1,14220.0,529864Vi73738,NaN,74.98,5.9984,1.8745,82.8529,None,D7395C0E-00CB-4BFA-A238-0D6A9F49884F,2014-07-07
31462,75122,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75122,None,10-4030-015868,15868,NaN,6,14616,14616,1,18719.0,330022Vi97312,NaN,30.97,2.4776,0.7743,34.2219,None,4221035A-4159-492F-AF40-4363A64FFC16,2014-07-07
31463,75123,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75123,None,10-4030-018759,18759,NaN,6,14024,14024,1,10084.0,230370Vi51970,NaN,189.97,15.1976,4.7493,209.9169,None,D54752FF-2B54-4BE5-95EA-3B72289C059F,2014-07-07


In [185]:
# 52 Retrieve the top 5 customers who have the longest average time between placing an order and receiving it.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [186]:
soh['diff'] = (pd.to_datetime(soh['DueDate']) - pd.to_datetime(soh['OrderDate']))

In [187]:
soh.groupby('CustomerID')['diff'].mean().reset_index().head()

,CustomerID,diff
0,11000,12 days
1,11001,12 days
2,11002,12 days
3,11003,12 days
4,11004,12 days


In [188]:
# 53 Find the total sales amount for each month of the year 2023, including orders that haven't been shipped yet

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [189]:
soh = soh[(pd.to_datetime(soh['ShipDate']).dt.year == 2014) | (soh['ShipDate'].isnull())]

In [190]:
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
19091,62751,8,2013-12-25,2014-01-06,2014-01-01,5,True,SO62751,None,10-4030-011932,11932,NaN,4,21734,21734,1,234.0,716911Vi1197,NaN,93.97,7.5176,2.3493,103.8369,None,27F9D1C7-8F7E-4623-87F7-11DD3AD4A1AB,2014-01-01
19092,62752,8,2013-12-25,2014-01-06,2014-01-01,5,True,SO62752,None,10-4030-015241,15241,NaN,9,17341,17341,1,17353.0,1217201Vi90144,10699.0,28.98,2.3184,0.7245,32.0229,None,04B884E7-8BF1-4604-8EC8-2B4411CC8300,2014-01-01
19093,62753,8,2013-12-25,2014-01-06,2014-01-01,5,True,SO62753,None,10-4030-014008,14008,NaN,9,26481,26481,1,18828.0,517233Vi97826,10699.0,28.98,2.3184,0.7245,32.0229,None,78C6D53B-CB64-4E15-8FB1-38D0ABDEE2DB,2014-01-01
19094,62754,8,2013-12-25,2014-01-06,2014-01-01,5,True,SO62754,None,10-4030-016632,16632,NaN,9,20331,20331,1,15424.0,817348Vi79667,10699.0,29.99,2.3992,0.7498,33.1390,None,0213CA89-EDDC-4DD1-A652-6D013461C4C1,2014-01-01
19095,62755,8,2013-12-25,2014-01-06,2014-01-01,5,True,SO62755,None,10-4030-018705,18705,NaN,9,14203,14203,1,14917.0,917562Vi77279,10699.0,56.97,4.5576,1.4243,62.9519,None,2B97A3FD-52A3-47A8-B78D-2C337CBC4B3E,2014-01-01


In [191]:
soh.groupby('CustomerID')['TotalDue'].sum().reset_index().head()

,CustomerID,TotalDue
0,11001,650.8008
1,11012,6.9394
2,11013,82.8529
3,11017,820.2968
4,11018,874.4086


In [192]:
# 54 Retrieve the orders placed by customers who have placed orders on consecutive days.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [193]:
# Convert 'OrderDate' to datetime
soh['OrderDate'] = pd.to_datetime(soh['OrderDate'])

In [194]:
# Sort the DataFrame by 'CustomerID' and 'OrderDate'
soh = soh.sort_values(by=['CustomerID', 'OrderDate'])
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
133,43793,8,2011-06-21,2011-07-03,2011-06-28,5,True,SO43793,None,10-4030-011000,11000,NaN,9,22601,22601,1,11981.0,735090Vi61907,275.0,3399.99,271.9992,84.9998,3756.9890,None,A95BFAFE-7543-43F7-A890-9AD41D4E6E5E,2011-06-28
7862,51522,8,2013-06-20,2013-07-02,2013-06-27,5,True,SO51522,None,10-4030-011000,11000,NaN,9,22601,22601,1,11981.0,739809Vi61907,8631.0,2341.97,187.3576,58.5493,2587.8769,None,D62326B5-58E9-4A12-848A-842C69B9E267,2013-06-27
13758,57418,8,2013-10-03,2013-10-15,2013-10-10,5,True,SO57418,None,10-4030-011000,11000,NaN,9,22601,22601,1,11981.0,738977Vi61907,9786.0,2507.03,200.5624,62.6758,2770.2682,None,62393E48-A72A-4C3C-A341-1B093BAA068D,2013-10-10
107,43767,8,2011-06-17,2011-06-29,2011-06-24,5,True,SO43767,None,10-4030-011001,11001,NaN,9,14489,14489,1,8810.0,235091Vi45462,223.0,3374.99,269.9992,84.3748,3729.3640,None,BF2E1105-44C2-422F-92E8-59C575BF2652,2011-06-24
7833,51493,8,2013-06-18,2013-06-30,2013-06-25,5,True,SO51493,None,10-4030-011001,11001,NaN,9,14489,14489,1,8810.0,239517Vi45462,8609.0,2419.93,193.5944,60.4983,2674.0227,None,A4A9383A-84B6-4500-8428-96CDB95DB1DC,2013-06-25


In [195]:
# Use shift to get the previous order date for each customer
soh['PrevOrderDate'] = soh.groupby('CustomerID')['OrderDate'].shift(1)
soh[['SalesOrderID', 'CustomerID', 'OrderDate', 'PrevOrderDate']].tail(25)

,SalesOrderID,CustomerID,OrderDate,PrevOrderDate
25809,69469,30115,2014-03-31,2013-12-31
7446,51106,30116,2013-05-30,NaT
11596,55256,30116,2013-08-30,2013-05-30
17528,61188,30116,2013-11-30,2013-08-30
23619,67279,30116,2014-03-01,2013-11-30
205,43865,30117,2011-07-01,NaT
849,44509,30117,2011-10-01,2011-07-01
1631,45291,30117,2012-01-01,2011-10-01
2389,46049,30117,2012-03-30,2012-01-01
3304,46964,30117,2012-06-30,2012-03-30


In [196]:
# Calculate the difference in days between consecutive orders
soh['OrderDateDiff'] = (soh['OrderDate'] - soh['PrevOrderDate']).dt.days
soh[['SalesOrderID', 'CustomerID', 'OrderDate', 'PrevOrderDate','OrderDateDiff']].tail()

,SalesOrderID,CustomerID,OrderDate,PrevOrderDate,OrderDateDiff
7015,50675,30118,2013-04-30,2013-01-28,92.0
9820,53480,30118,2013-07-31,2013-04-30,92.0
15268,58928,30118,2013-10-30,2013-07-31,91.0
21561,65221,30118,2014-01-29,2013-10-30,91.0
28143,71803,30118,2014-05-01,2014-01-29,92.0


In [197]:
# Identify customers who have at least one pair of consecutive orders (difference of 1 day)
consecutive_customers = soh[soh['OrderDateDiff'] == 1]['CustomerID'].unique()
consecutive_customers

array([11019, 11078, 11091, 11142, 11176, 11185, 11203, 11215, 11262,
       11276, 11277, 11287, 11300, 11330, 11498, 11500, 11501, 11502,
       11506, 11566, 11619, 11631, 11669, 11711, 11861, 11863, 12034,
       12323, 12532, 12936, 13614, 14265, 14332, 15124, 15172, 17307,
       17374, 17807, 18274, 18755, 29586, 29676, 29679, 29745, 29812,
       29837, 30079], dtype=int64)

In [198]:
# Retrieve all orders for those customers
consecutive_orders = soh[soh['CustomerID'].isin(consecutive_customers)].drop(columns=['PrevOrderDate', 'OrderDateDiff'])
consecutive_orders.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
8966,52626,8,2013-07-15,2013-07-27,2013-07-22,5,True,SO52626,None,10-4030-011019,11019,NaN,6,14167,14167,1,10087.0,121109Vi51988,8908.0,37.97,3.0376,0.9493,41.9569,None,0567630F-D2E6-4090-8259-18C33F1EEA2C,2013-07-22
10174,53834,8,2013-08-04,2013-08-16,2013-08-11,5,True,SO53834,None,10-4030-011019,11019,NaN,6,14167,14167,1,10087.0,120585Vi51988,9128.0,53.99,4.3192,1.3498,59.6590,None,CF43CAD5-316F-4E0C-AE0B-50BF431154BC,2013-08-11
10672,54332,8,2013-08-13,2013-08-25,2013-08-20,5,True,SO54332,None,10-4030-011019,11019,NaN,6,14167,14167,1,10087.0,120838Vi51988,9227.0,47.97,3.8376,1.1993,53.0069,None,D646A94A-9F24-499F-9784-B71B3D438E3D,2013-08-20
13250,56910,8,2013-09-28,2013-10-10,2013-10-05,5,True,SO56910,None,10-4030-011019,11019,NaN,6,14167,14167,1,10087.0,116741Vi51988,9733.0,8.99,0.7192,0.2248,9.9340,None,D336FAE4-05F0-4EDF-966D-E3B0B1BBF9CA,2013-10-05
13979,57639,8,2013-10-08,2013-10-20,2013-10-15,5,True,SO57639,None,10-4030-011019,11019,NaN,6,14167,14167,1,10087.0,121283Vi51988,9843.0,7.28,0.5824,0.1820,8.0444,None,142B6F95-55F2-47B9-A6C1-D7533C609784,2013-10-15


In [199]:
# 55 List all orders placed on holidays (e.g., New Year's Day, Christmas Day).



In [200]:
# 56 Retrieve the orders placed on weekdays (Monday to Friday) where the order was shipped on the next business day.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [201]:
soh[(soh['OrderDate'].dt.weekday>=1) & (soh['OrderDate'].dt.weekday<=5) & (soh['OrderDate'] - soh['ShipDate'] == 1)].head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate


In [202]:
# 57 Find the total sales amount for each weekday (Monday to Sunday) for the year 2023.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [203]:
soh = soh[(soh['OrderDate'].dt.weekday>=0) & (soh['OrderDate'].dt.weekday<=6) & (soh['OrderDate'].dt.year==2014)]

In [204]:
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
19703,63363,8,2014-01-01,2014-01-13,2014-01-08,5,True,SO63363,None,10-4030-014685,14685,NaN,9,23543,23543,1,6121.0,116821Vi31804,10776.0,27.28,2.1824,0.6820,30.1444,None,5DC63392-A8FD-4AF4-B061-6E2D37E59BA9,2014-01-08
19704,63364,8,2014-01-01,2014-01-13,2014-01-08,5,True,SO63364,None,10-4030-018299,18299,NaN,9,22550,22550,1,12333.0,117556Vi63808,10776.0,29.93,2.3944,0.7483,33.0727,None,48EE86B4-DC4E-40CB-BE34-58CCD327EA73,2014-01-08
19705,63365,8,2014-01-01,2014-01-13,2014-01-08,5,True,SO63365,None,10-4030-021279,21279,NaN,9,28760,28760,1,4739.0,317865Vi24853,10776.0,4.99,0.3992,0.1248,5.5140,None,CDD1967C-37E9-4F47-B9AB-5BF9AE04178E,2014-01-08
19706,63366,8,2014-01-01,2014-01-13,2014-01-08,5,True,SO63366,None,10-4030-014518,14518,NaN,9,16300,16300,1,12130.0,118418Vi62713,10776.0,62.98,5.0384,1.5745,69.5929,None,39098369-FAF9-4467-8A1B-878B78CF48BC,2014-01-08
19707,63367,8,2014-01-01,2014-01-13,2014-01-08,5,True,SO63367,None,10-4030-013605,13605,NaN,7,27067,27067,1,8488.0,219707Vi43885,NaN,2478.34,198.2672,61.9585,2738.5657,None,42C5AA28-62AE-4E67-B2DC-A9541B0781BD,2014-01-08


In [205]:
soh.groupby(soh['OrderDate'].dt.day_name())['TotalDue'].sum().reset_index()

,OrderDate,TotalDue
0,Friday,1.231253e+06
1,Monday,5.117403e+06
2,Saturday,3.702891e+06
3,Sunday,1.396873e+06
4,Thursday,5.190897e+06
5,Tuesday,1.300207e+06
6,Wednesday,4.479975e+06


In [206]:
# 58 Retrieve the orders placed by customers who haven't placed any orders in the year 2023.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [207]:
soh = soh[soh['OrderDate'].dt.year == 2014]
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
19703,63363,8,2014-01-01,2014-01-13,2014-01-08,5,True,SO63363,None,10-4030-014685,14685,NaN,9,23543,23543,1,6121.0,116821Vi31804,10776.0,27.28,2.1824,0.6820,30.1444,None,5DC63392-A8FD-4AF4-B061-6E2D37E59BA9,2014-01-08
19704,63364,8,2014-01-01,2014-01-13,2014-01-08,5,True,SO63364,None,10-4030-018299,18299,NaN,9,22550,22550,1,12333.0,117556Vi63808,10776.0,29.93,2.3944,0.7483,33.0727,None,48EE86B4-DC4E-40CB-BE34-58CCD327EA73,2014-01-08
19705,63365,8,2014-01-01,2014-01-13,2014-01-08,5,True,SO63365,None,10-4030-021279,21279,NaN,9,28760,28760,1,4739.0,317865Vi24853,10776.0,4.99,0.3992,0.1248,5.5140,None,CDD1967C-37E9-4F47-B9AB-5BF9AE04178E,2014-01-08
19706,63366,8,2014-01-01,2014-01-13,2014-01-08,5,True,SO63366,None,10-4030-014518,14518,NaN,9,16300,16300,1,12130.0,118418Vi62713,10776.0,62.98,5.0384,1.5745,69.5929,None,39098369-FAF9-4467-8A1B-878B78CF48BC,2014-01-08
19707,63367,8,2014-01-01,2014-01-13,2014-01-08,5,True,SO63367,None,10-4030-013605,13605,NaN,7,27067,27067,1,8488.0,219707Vi43885,NaN,2478.34,198.2672,61.9585,2738.5657,None,42C5AA28-62AE-4E67-B2DC-A9541B0781BD,2014-01-08


In [208]:
cus = db_reader().read_table('sales.customer', connection)
cus.head()

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,2,NaN,1028.0,1,AW00000002,E552F657-A9AF-4A7D-A645-C429D6E02491,2014-09-12 11:15:07.263
1,3,NaN,642.0,4,AW00000003,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2014-09-12 11:15:07.263
2,4,NaN,932.0,4,AW00000004,FF862851-1DAA-4044-BE7C-3E85583C054D,2014-09-12 11:15:07.263
3,5,NaN,1026.0,4,AW00000005,83905BDC-6F5E-4F71-B162-C98DA069F38A,2014-09-12 11:15:07.263
4,6,NaN,644.0,4,AW00000006,1A92DF88-BFA2-467D-BD54-FCB9E647FDD7,2014-09-12 11:15:07.263


In [209]:
soh[~soh['CustomerID'].isin(cus['CustomerID'])].head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate


In [210]:
# 59 Find the average time taken to ship orders for each product category.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [211]:
sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [212]:
d = merge().merge_tbl(soh[['SalesOrderID', 'OrderDate','ShipDate']], sod[['SalesOrderID', 'ProductID']], 'inner', 'SalesOrderID')
d.head()

,SalesOrderID,OrderDate,ShipDate,ProductID
0,43660,2011-05-20,2011-06-07,762
1,43660,2011-05-20,2011-06-07,758
2,43661,2011-05-31,2011-06-07,745
3,43661,2011-05-31,2011-06-07,743
4,43661,2011-05-31,2011-06-07,747


In [213]:
p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [214]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [215]:
c = merge().merge_tbl(p[['ProductID','ProductSubcategoryID']], psc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner', 'ProductSubcategoryID')
c.head()

,ProductID,ProductSubcategoryID,ProductCategoryID
0,680,14.0,2
1,706,14.0,2
2,707,31.0,4
3,708,31.0,4
4,709,23.0,3


In [216]:
id = merge().merge_tbl(d, c[['ProductID', 'ProductCategoryID']], 'inner', 'ProductID')
id.head()

,SalesOrderID,OrderDate,ShipDate,ProductID,ProductCategoryID
0,43660,2011-05-20,2011-06-07,762,1
1,43660,2011-05-20,2011-06-07,758,1
2,43661,2011-05-31,2011-06-07,745,2
3,43661,2011-05-31,2011-06-07,743,2
4,43661,2011-05-31,2011-06-07,747,2


In [217]:
id['avg'] = (id['ShipDate'] - id['OrderDate']).dt.days.mean()

In [218]:
id.head()

,SalesOrderID,OrderDate,ShipDate,ProductID,ProductCategoryID,avg
0,43660,2011-05-20,2011-06-07,762,1,7.000412
1,43660,2011-05-20,2011-06-07,758,1,7.000412
2,43661,2011-05-31,2011-06-07,745,2,7.000412
3,43661,2011-05-31,2011-06-07,743,2,7.000412
4,43661,2011-05-31,2011-06-07,747,2,7.000412


In [219]:
# 60 Retrieve the orders placed on the last business day of each month.



In [220]:
# 61 Find the number of orders placed in each hour of the day, considering only orders placed between 9:00 AM and 5:00 PM.


In [221]:
# 62 Retrieve the top 3 customers who have the highest total sales amount in each quarter of the year 2023.

In [222]:
# 63 Find the average time taken to ship orders for each customer, considering only orders placed in the second half of the year.

In [223]:
# 64 Retrieve the orders placed by customers who have placed orders on all weekdays (Monday to Friday) of the week.

In [224]:
# 65 List all orders placed on the first business day of each month.

In [225]:
# 66 Retrieve the orders placed on weekends (Saturday or Sunday) where the order was shipped on the next business day.


In [226]:
# 67 Find the total sales amount for each product category for orders placed on the first Monday of each month.


In [227]:
# 68 Retrieve the orders placed on the same day of the week as the current date

In [228]:
# 69 Find the average number of days between placing an order and shipping it, grouped by the year and month of the order date.

In [229]:
# 70 Retrieve the orders placed by customers who have not placed any orders in the last 90 days.

In [230]:
# 71 Find the total sales amount for each hour of the day, considering only orders placed on weekdays (Monday to Friday).

In [231]:
# 72 Retrieve the first name and last name of employees, concatenating them with a space in between.

per = db_reader().read_table('person.person', connection)
per.head()


,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30


In [232]:
per['Name']=per['FirstName'] + ' ' + per['LastName']

In [233]:
per[['BusinessEntityID', 'Name']].head()

,BusinessEntityID,Name
0,1,Ken Sánchez
1,2,Terri Duffy
2,3,Roberto Tamburello
3,4,Rob Walters
4,5,Gail Erickson


In [234]:
# 73 List all products whose names contain the word "bike".

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [235]:
pro[pro['Name'].str.contains('bike', case=False)].head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
213,709,"Mountain Bike Socks, M",SO-B909-M,False,True,White,4,3,3.3963,9.50,M,None,None,NaN,0,M,None,U,23.0,18.0,2011-05-31,2012-05-29,NaT,18F95F47-1540-4E02-8F1F-CC1BCB6828D0,2014-02-08 10:01:36.827
214,710,"Mountain Bike Socks, L",SO-B909-L,False,True,White,4,3,3.3963,9.50,L,None,None,NaN,0,M,None,U,23.0,18.0,2011-05-31,2012-05-29,NaT,161C035E-21B3-4E14-8E44-AF508F35D80A,2014-02-08 10:01:36.827
380,876,Hitch Rack - 4-Bike,RA-H123,False,True,None,4,3,44.8800,120.00,None,None,None,NaN,0,S,None,None,26.0,118.0,2013-05-30,NaT,NaT,7A0C4BBD-9679-4F59-9EBC-9DAF3439A38A,2014-02-08 10:01:36.827
381,877,Bike Wash - Dissolver,CL-9009,False,True,None,4,3,2.9733,7.95,None,None,None,NaN,0,S,None,None,29.0,119.0,2013-05-30,NaT,NaT,3C40B5AD-E328-4715-88A7-EC3220F02ACF,2014-02-08 10:01:36.827
383,879,All-Purpose Bike Stand,ST-1401,False,True,None,4,3,59.4660,159.00,None,None,None,NaN,0,M,None,None,27.0,122.0,2013-05-30,NaT,NaT,C7BB564B-A637-40F5-B21B-CBF7E4F713BE,2014-02-08 10:01:36.827


In [236]:
# 74 Retrieve the email addresses of customers, ensuring they are in lowercase.

email = db_reader().read_table('person.emailaddress', connection)
email.head()

,BusinessEntityID,EmailAddressID,EmailAddress,rowguid,ModifiedDate
0,1,1,newmail@gmail.com,8A1901E4-671B-431A-871C-EADB2942E9EE,2009-01-07
1,2,2,terri0@adventure-works.com,B5FF9EFD-72A2-4F87-830B-F338FDD4D162,2008-01-24
2,3,3,roberto0@adventure-works.com,C8A51084-1C03-4C58-A8B3-55854AE7C499,2007-11-04
3,4,4,rob0@adventure-works.com,17703ED1-0031-4B4A-AFD2-77487A556B3B,2007-11-28
4,5,5,gail0@adventure-works.com,E76D2EA3-08E5-409C-BBE2-5DD1CDF89A3B,2007-12-30


In [237]:
email[email['EmailAddress'].str.islower()].head()

,BusinessEntityID,EmailAddressID,EmailAddress,rowguid,ModifiedDate
0,1,1,newmail@gmail.com,8A1901E4-671B-431A-871C-EADB2942E9EE,2009-01-07
1,2,2,terri0@adventure-works.com,B5FF9EFD-72A2-4F87-830B-F338FDD4D162,2008-01-24
2,3,3,roberto0@adventure-works.com,C8A51084-1C03-4C58-A8B3-55854AE7C499,2007-11-04
3,4,4,rob0@adventure-works.com,17703ED1-0031-4B4A-AFD2-77487A556B3B,2007-11-28
4,5,5,gail0@adventure-works.com,E76D2EA3-08E5-409C-BBE2-5DD1CDF89A3B,2007-12-30


In [238]:
# 75 Find the length of the product names.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [239]:
pro['Name'].str.len().head()

0    15
1    12
2    15
3    21
4     5
Name: Name, dtype: int64

In [240]:
# 76 List all customers whose names start with the letter "A".

per = db_reader().read_table('person.person', connection)
per.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30


In [241]:
per['Name']=per['FirstName'] + ' ' + per['LastName']

In [242]:
per[per['Name'].str.startswith('A')].head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate,Name
32,33,EM,False,None,Annik,O,Stahl,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",BFA2BDC8-208A-447B-8A4E-72A215E9E134,2008-12-10,Annik Stahl
46,47,EM,False,None,Andrew,R,Hill,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",B6439F4F-E57D-4293-B0C8-8919591055A3,2009-02-15,Andrew Hill
108,109,EM,False,None,Alice,O,Ciccu,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",B5836E66-4251-4D4D-8F49-8E6D9E3C9D64,2008-11-30,Alice Ciccu
114,115,EM,False,None,Angela,W,Barbariol,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",8E6ADFD0-2FE0-4EB9-93F2-A04FD877F6DC,2009-01-13,Angela Barbariol
136,137,EM,False,None,Anibal,T,Sousa,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",9B75A538-958C-4C91-987A-B9BCCE683E8A,2009-02-16,Anibal Sousa


In [243]:
# 77 Retrieve the first 3 characters of each product's name

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [244]:
pro['Name'].str[:3].head()

0    Adj
1    Bea
2    BB 
3    Hea
4    Bla
Name: Name, dtype: object

In [245]:
# 78 Find the position of the first occurrence of the letter "o" in each product's name.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [246]:
pro['Name'].str.find('o').tail()

501     1
502     1
503     1
504    21
505    -1
Name: Name, dtype: int64

In [247]:
# 79 List all customers whose names contain exactly 5 characters.

per = db_reader().read_table('person.person', connection)
per.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30


In [248]:
per[per['FirstName'].str.len() == 5].head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
6,7,EM,False,None,Dylan,A,Miller,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",C45E8AB8-01BE-4B76-B215-820C8368181A,2009-02-01
7,8,EM,False,None,Diane,L,Margheim,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",A948E590-4A56-45A9-BC9A-160A1CC9D990,2008-12-22
15,16,EM,False,None,David,M,Bradley,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",2CC8BA72-5DBB-497F-B941-775517993638,2007-12-13
16,17,EM,False,None,Kevin,F,Brown,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",9EE4713E-B3D8-4409-BC5E-DEC62497F43A,2007-01-19


In [249]:
# 80 Retrieve the last 4 characters of each product's name.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [250]:
pro['Name'].str[-4:].head()

0    Race
1    Ball
2    ring
3    ings
4    lade
Name: Name, dtype: object

In [251]:
# 81 Find the product names in uppercase.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [252]:
pro[pro['Name'].str.isupper()].head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
505,2006,BIKE HANDLE,RV-H123,True,True,BLUE,250,20,145.88,2000.0,20,CM,LB,2.0,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,D67538F2-A81A-4A4F-98F7-202002D888B4,2024-03-02 14:57:55.763


In [253]:
# 82 Retrieve the first name and last name of employees, ensuring that each name starts with an uppercase letter followed by lowercase letters.

per = db_reader().read_table('person.person', connection)
per.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30


In [254]:
per['FirstName'].str.capitalize().head()

0        Ken
1      Terri
2    Roberto
3        Rob
4       Gail
Name: FirstName, dtype: object

In [255]:
# 83 List all products whose names are longer than the average length of product names.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [256]:
avg = pro['Name'].str.len().mean()
avg

18.43083003952569

In [257]:
pro[pro['Name'].str.len() > avg].head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
31,351,Front Derailleur Cage,FC-3982,False,False,Silver,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,01C901E3-4323-48ED-AB9E-9BFDA28BDEF6,2014-02-08 10:01:36.827
32,352,Front Derailleur Linkage,FL-2301,False,False,Silver,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,88ED2E08-E775-4915-B506-831600B773FD,2014-02-08 10:01:36.827
38,360,Thin-Jam Hex Nut 10,HJ-1220,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,A1AE0C6D-92A5-4FDA-B33B-1301E83EFE5B,2014-02-08 10:01:36.827
41,363,Thin-Jam Hex Nut 15,HJ-3410,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,B9D9A30D-CB07-422D-A312-F0535575337E,2014-02-08 10:01:36.827


In [258]:
# 84 Retrieve the email addresses of customers, splitting them into username and domain parts.

email = db_reader().read_table('person.emailaddress', connection)
email.head()

,BusinessEntityID,EmailAddressID,EmailAddress,rowguid,ModifiedDate
0,1,1,newmail@gmail.com,8A1901E4-671B-431A-871C-EADB2942E9EE,2009-01-07
1,2,2,terri0@adventure-works.com,B5FF9EFD-72A2-4F87-830B-F338FDD4D162,2008-01-24
2,3,3,roberto0@adventure-works.com,C8A51084-1C03-4C58-A8B3-55854AE7C499,2007-11-04
3,4,4,rob0@adventure-works.com,17703ED1-0031-4B4A-AFD2-77487A556B3B,2007-11-28
4,5,5,gail0@adventure-works.com,E76D2EA3-08E5-409C-BBE2-5DD1CDF89A3B,2007-12-30


In [259]:
id = email[email['EmailAddress'].str.contains('@')]
id.head()

,BusinessEntityID,EmailAddressID,EmailAddress,rowguid,ModifiedDate
0,1,1,newmail@gmail.com,8A1901E4-671B-431A-871C-EADB2942E9EE,2009-01-07
1,2,2,terri0@adventure-works.com,B5FF9EFD-72A2-4F87-830B-F338FDD4D162,2008-01-24
2,3,3,roberto0@adventure-works.com,C8A51084-1C03-4C58-A8B3-55854AE7C499,2007-11-04
3,4,4,rob0@adventure-works.com,17703ED1-0031-4B4A-AFD2-77487A556B3B,2007-11-28
4,5,5,gail0@adventure-works.com,E76D2EA3-08E5-409C-BBE2-5DD1CDF89A3B,2007-12-30


In [260]:
id[['Username', 'Domain']] = id['EmailAddress'].str.split('@', expand=True)

In [261]:
id.head()

,BusinessEntityID,EmailAddressID,EmailAddress,rowguid,ModifiedDate,Username,Domain
0,1,1,newmail@gmail.com,8A1901E4-671B-431A-871C-EADB2942E9EE,2009-01-07,newmail,gmail.com
1,2,2,terri0@adventure-works.com,B5FF9EFD-72A2-4F87-830B-F338FDD4D162,2008-01-24,terri0,adventure-works.com
2,3,3,roberto0@adventure-works.com,C8A51084-1C03-4C58-A8B3-55854AE7C499,2007-11-04,roberto0,adventure-works.com
3,4,4,rob0@adventure-works.com,17703ED1-0031-4B4A-AFD2-77487A556B3B,2007-11-28,rob0,adventure-works.com
4,5,5,gail0@adventure-works.com,E76D2EA3-08E5-409C-BBE2-5DD1CDF89A3B,2007-12-30,gail0,adventure-works.com


In [262]:
# 85 Find the product names along with the number of words in each name.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [263]:
pro['WordCount'] = pro['Name'].apply(lambda x: len(str(x).split()))

In [264]:
pro['WordCount2'] = pro['Name'].str.split().str.len()

In [265]:
pro[['Name', 'WordCount', 'WordCount2']].head()

,Name,WordCount,WordCount2
0,Adjustable Race,2,2
1,Bearing Ball,2,2
2,BB Ball Bearing,3,3
3,Headset Ball Bearings,3,3
4,Blade,1,1


In [266]:
# 86 List all customers whose names contain at least two words.

per = db_reader().read_table('person.person', connection)
per.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30


In [267]:
per['Name']=per['FirstName'] + ' ' + per['LastName']

In [268]:
p = per[['BusinessEntityID', 'Name']]
p.head()

,BusinessEntityID,Name
0,1,Ken Sánchez
1,2,Terri Duffy
2,3,Roberto Tamburello
3,4,Rob Walters
4,5,Gail Erickson


In [269]:
p[p['Name'].str.split().str.len() >= 2].head()

,BusinessEntityID,Name
0,1,Ken Sánchez
1,2,Terri Duffy
2,3,Roberto Tamburello
3,4,Rob Walters
4,5,Gail Erickson


In [270]:
# 87 Retrieve the product names with the last word removed.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [271]:
name = pro[['Name']]
name.head()

,Name
0,Adjustable Race
1,Bearing Ball
2,BB Ball Bearing
3,Headset Ball Bearings
4,Blade


In [272]:
name['LastWordRemoved'] = name['Name'].str.split().str[:-1].str.join('')

In [273]:
name.head()

,Name,LastWordRemoved
0,Adjustable Race,Adjustable
1,Bearing Ball,Bearing
2,BB Ball Bearing,BBBall
3,Headset Ball Bearings,HeadsetBall
4,Blade,


In [274]:
# 88 Find the position of the second occurrence of the letter "e" in each product's name

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [275]:
def find_second_e_position(name):
    first_pos = name.find('e')
    if first_pos == -1:
        return -1  # Return -1 if 'e' is not found at all
    second_pos = name.find('e', first_pos + 1)
    if second_pos == -1:
        return -1  # Return -1 if 'e' is not found again after the first occurrence
    return second_pos

In [276]:
pro['SecondEPosition'] = pro['Name'].apply(find_second_e_position)

In [277]:
pro[['Name', 'SecondEPosition']].head()

,Name,SecondEPosition
0,Adjustable Race,14
1,Bearing Ball,-1
2,BB Ball Bearing,-1
3,Headset Ball Bearings,5
4,Blade,-1


In [278]:
# 89 List all customers whose names end with the letter "n" and have exactly two words in their names.

per = db_reader().read_table('person.person', connection)
per.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30


In [279]:
per['Name']=per['FirstName'] + ' ' + per['LastName']

In [280]:
per[(per['Name'].str.endswith('n')) & (per['Name'].str.split().str.len() >=2)].head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate,Name
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30,Gail Erickson
12,13,EM,False,Ms.,Janice,M,Galvin,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",34EB99E0-7042-4DC1-A2FD-BDA290FF0E07,2010-12-16,Janice Galvin
13,14,EM,False,None,Michael,I,Sullivan,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",9A7501DE-5CAF-4700-AB07-CC81102BB696,2010-12-23,Michael Sullivan
16,17,EM,False,None,Kevin,F,Brown,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",9EE4713E-B3D8-4409-BC5E-DEC62497F43A,2007-01-19,Kevin Brown
22,23,EM,False,None,Mary,E,Gibson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",B10CF8EB-5589-4A81-B1BD-051C5759CE06,2009-01-05,Mary Gibson


In [281]:
# 90 Retrieve the product names sorted in descending order based on the number of vowels in each name.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [282]:
pro['Count'] = pro['Name'].str.count('[aeiouAEIOU]')

In [283]:
pro[['Name', 'Count']].sort_values('Count', ascending=False).head()

,Name,Count
504,Vintage Handcrafted Wooden Handlebars,12
208,Rear Derailleur Cage,9
32,Front Derailleur Linkage,9
329,HL Mountain Rear Wheel,8
422,"LL Mountain Frame - Silver, 44",8


In [284]:
# 91 Retrieve the product names with the words reversed within each name.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [285]:
pro['ReversedName'] = pro['Name'].str[::-1]

In [286]:
pro[['Name', 'ReversedName']].head()

,Name,ReversedName
0,Adjustable Race,ecaR elbatsujdA
1,Bearing Ball,llaB gniraeB
2,BB Ball Bearing,gniraeB llaB BB
3,Headset Ball Bearings,sgniraeB llaB tesdaeH
4,Blade,edalB


In [287]:
# 92 Retrieve the product names along with the number of vowels in each name.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [288]:
pro['Count'] = pro['Name'].str.count('[aeiouAEIOU]')

In [289]:
pro[['Name', 'Count']].sort_values('Count', ascending=False).head()

,Name,Count
504,Vintage Handcrafted Wooden Handlebars,12
208,Rear Derailleur Cage,9
32,Front Derailleur Linkage,9
329,HL Mountain Rear Wheel,8
422,"LL Mountain Frame - Silver, 44",8


In [290]:
# 93 List all customers whose email addresses contain a domain ending with ".com".

email = db_reader().read_table('person.emailaddress', connection)
email.head()

,BusinessEntityID,EmailAddressID,EmailAddress,rowguid,ModifiedDate
0,1,1,newmail@gmail.com,8A1901E4-671B-431A-871C-EADB2942E9EE,2009-01-07
1,2,2,terri0@adventure-works.com,B5FF9EFD-72A2-4F87-830B-F338FDD4D162,2008-01-24
2,3,3,roberto0@adventure-works.com,C8A51084-1C03-4C58-A8B3-55854AE7C499,2007-11-04
3,4,4,rob0@adventure-works.com,17703ED1-0031-4B4A-AFD2-77487A556B3B,2007-11-28
4,5,5,gail0@adventure-works.com,E76D2EA3-08E5-409C-BBE2-5DD1CDF89A3B,2007-12-30


In [291]:
email[email['EmailAddress'].str.endswith('.com')].head()

,BusinessEntityID,EmailAddressID,EmailAddress,rowguid,ModifiedDate
0,1,1,newmail@gmail.com,8A1901E4-671B-431A-871C-EADB2942E9EE,2009-01-07
1,2,2,terri0@adventure-works.com,B5FF9EFD-72A2-4F87-830B-F338FDD4D162,2008-01-24
2,3,3,roberto0@adventure-works.com,C8A51084-1C03-4C58-A8B3-55854AE7C499,2007-11-04
3,4,4,rob0@adventure-works.com,17703ED1-0031-4B4A-AFD2-77487A556B3B,2007-11-28
4,5,5,gail0@adventure-works.com,E76D2EA3-08E5-409C-BBE2-5DD1CDF89A3B,2007-12-30


In [292]:
# 94 Retrieve the first name and last name of employees, reversing their names.

per = db_reader().read_table('person.person', connection)
per.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30


In [293]:
per['Name']=per['FirstName'] + ' ' + per['LastName']

In [294]:
per['RName'] = per['Name'].str[::-1]

In [295]:
per[['FirstName' ,'LastName', 'RName']].head()

,FirstName,LastName,RName
0,Ken,Sánchez,zehcnáS neK
1,Terri,Duffy,yffuD irreT
2,Roberto,Tamburello,ollerubmaT otreboR
3,Rob,Walters,sretlaW boR
4,Gail,Erickson,noskcirE liaG


In [296]:
# 95 Find the product names along with the number of characters in each name excluding spaces.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [297]:
pro['Len'] = pro['Name'].str.replace(' ', '').str.len()

In [298]:
pro[['Name', 'Len']].head()

,Name,Len
0,Adjustable Race,14
1,Bearing Ball,11
2,BB Ball Bearing,13
3,Headset Ball Bearings,19
4,Blade,5


In [299]:
# 96 Retrieve the email addresses of customers, extracting the domain name part.

email = db_reader().read_table('person.emailaddress', connection)
email.head()

,BusinessEntityID,EmailAddressID,EmailAddress,rowguid,ModifiedDate
0,1,1,newmail@gmail.com,8A1901E4-671B-431A-871C-EADB2942E9EE,2009-01-07
1,2,2,terri0@adventure-works.com,B5FF9EFD-72A2-4F87-830B-F338FDD4D162,2008-01-24
2,3,3,roberto0@adventure-works.com,C8A51084-1C03-4C58-A8B3-55854AE7C499,2007-11-04
3,4,4,rob0@adventure-works.com,17703ED1-0031-4B4A-AFD2-77487A556B3B,2007-11-28
4,5,5,gail0@adventure-works.com,E76D2EA3-08E5-409C-BBE2-5DD1CDF89A3B,2007-12-30


In [300]:
email['Domain'] = email['EmailAddress'].str.extract(r'@(.*)')

In [301]:
email[['EmailAddress', 'Domain']].head()

,EmailAddress,Domain
0,newmail@gmail.com,gmail.com
1,terri0@adventure-works.com,adventure-works.com
2,roberto0@adventure-works.com,adventure-works.com
3,rob0@adventure-works.com,adventure-works.com
4,gail0@adventure-works.com,adventure-works.com


In [302]:
# 97 List all customers whose names contain a palindrome (a word that reads the same forwards and backwards).

per = db_reader().read_table('person.person', connection)
per.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30


In [303]:
per['Name']=per['FirstName'] + ' ' + per['LastName']

In [304]:
per['RName'] = per['Name'].str[::-1]

In [305]:
per[['Name', 'RName']].head()

,Name,RName
0,Ken Sánchez,zehcnáS neK
1,Terri Duffy,yffuD irreT
2,Roberto Tamburello,ollerubmaT otreboR
3,Rob Walters,sretlaW boR
4,Gail Erickson,noskcirE liaG


In [306]:
per[per['Name'] == per['RName']]

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate,Name,RName


In [307]:
# 98 Retrieve the product names with each word capitalized (title case).

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [308]:
pro['Name'].str.title().head()

0          Adjustable Race
1             Bearing Ball
2          Bb Ball Bearing
3    Headset Ball Bearings
4                    Blade
Name: Name, dtype: object

In [309]:
pro['Name'].str.upper().head()

0          ADJUSTABLE RACE
1             BEARING BALL
2          BB BALL BEARING
3    HEADSET BALL BEARINGS
4                    BLADE
Name: Name, dtype: object

In [310]:
# 99 List all customers whose names contain more vowels than consonants.

per = db_reader().read_table('person.person', connection)
per.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30


In [311]:
per['Name']=per['FirstName'] + ' ' + per['LastName']

In [312]:
per['Vowels'] = per['Name'].str.count('[aeiouAEIOU]')

In [313]:
per['Consonants'] = per['Name'].str.replace('[aeiouAEIOU]', '', regex=True).str.len()



In [314]:
a = per[['Name', 'Vowels', 'Consonants']]
a.tail()

,Name,Vowels,Consonants
19968,Isabella Richardson,7,12
19969,Crystal He,2,8
19970,Crystal Zheng,2,11
19971,Crystal Hu,2,8
19972,ARAVINDH A,4,6


In [315]:
a[per['Vowels'] > per['Consonants']].head()

,Name,Vowels,Consonants
271,Janaina Bueno,7,6
300,Samuel Agcaoili,8,7
321,Emilio Alvaro,7,6
616,Leo Giakoumakis,8,7
711,Lucio Iallo,6,5


In [316]:
# 100 Retrieve the product names along with the number of times the letter "s" appears in each name.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [317]:
pro['Count'] = pro['Name'].str.count('s')

In [318]:
pro[['Name', 'Count']].head()

,Name,Count
0,Adjustable Race,1
1,Bearing Ball,0
2,BB Ball Bearing,0
3,Headset Ball Bearings,2
4,Blade,0
